# DSAIT4335 Recommender Systems
# Final Project

In this project, you will work to build different recommendation models and evaluate the effectiveness of these models through offline experiments. The dataset used for the experiments is **MovieLens100K**, a movie recommendation dataset collected by GroupLens: https://grouplens.org/datasets/movielens/100k/. For more details, check the project description on Brightspace.

# Instruction

The MovieLens100K is already splitted into 80% training and 20% test sets. Along with training and test sets, movies metadata as content information is also provided.

**Expected file structure** for this assignment:   
   
   ```
   RecSysProject/
   ├── training.txt
   ├── test.txt
   ├── movies.txt
   └── codes.ipynb
   ```

**Note:** Be sure to run all cells in each section sequentially, so that intermediate variables and packages are properly carried over to subsequent cells.

**Note** Be sure to run all cells such that the submitted file contains the output of each cell.

**Note** Feel free to add cells if you need more for answering a question.

**Submission:** Answer all the questions in this jupyter-notebook file. Submit this jupyter-notebook file (your answers included) to Brightspace. Change the name of this jupyter-notebook file to your group number: example, group10 -> 10.ipynb.

# Setup

In [ ]:
!pip install -q transformers torch  # For BERT
!pip install numpy pandas matplotlib seaborn scipy scikit-learn torch tqdm
!pip install torch-directml

# you can refer https://huggingface.co/docs/transformers/en/model_doc/bert for various versions of the pre-trained model BERT

In [ ]:
# For BERT embeddings (install: pip install transformers torch)
print("Check the status of BERT installation:")

try:
    from transformers import AutoTokenizer, AutoModel
    import torch

    BERT_AVAILABLE = True
    directml_available = False
    device = None

    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        # Try DirectML (Windows / AMD) if installed
        try:
            import torch_directml
            dml_device = torch_directml.device()
            for i in range(torch_directml.device_count()):
                print(f"  Device {i}: {torch_directml.device_name(i)}")
            device = dml_device
            directml_available = True
        except Exception:
            # Try Apple MPS
            if getattr(torch.backends, "mps", None) is not None and torch.backends.mps.is_available():
                device = torch.device("mps")
            else:
                # Fallback to CPU
                device = torch.device("cpu")

    print("BERT libraries loaded successfully!")
    # device = torch.device('cuda' if torch.cuda.is_available() else 'mps')
    print("torch version:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available(), " CUDA devices:", torch.cuda.device_count() if torch.cuda.is_available() else 0)
    print("MPS available:", getattr(torch.backends, "mps", None) is not None and torch.backends.mps.is_available())
    print("DirectML available:", directml_available)
    print("Using device:", device)
except ImportError:
    BERT_AVAILABLE = False
    import torch
    device = torch.device("cpu")
    directml_available = False
    print("BERT libraries not available. Install with: pip install transformers torch")
    print("Using device:", device)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix
from scipy.spatial.distance import cosine, correlation
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from collections import defaultdict
import torch
import torch.nn as nn
import torch.optim as optim
import re
import time, math
from tqdm import tqdm
import random
import itertools
import json
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

### Here we set the random seed to 10 in order to ensure reproducibility of the experiments.

In [ ]:
SEED = 10

# Python built-in random seed
random.seed(SEED)

# Numpy random seed
np.random.seed(SEED)

# PyTorch random seed (CPU)
torch.manual_seed(SEED)

# PyTorch random seed (GPU, if available)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # if using multi-GPU

In [ ]:
# Compatibility for older numpy version
try:
    import numpy.core.numeric
    import sys
    sys.modules['numpy._core.numeric'] = numpy.core.numeric
except ImportError:
    pass

## Mount Data Folder

In [ ]:
import os

# Use current directory as project root
PROJECT_PATH = os.getcwd()  # e.g. /Users/yourname/Documents/Project-RecSys
OUTPUTS_PATH = os.path.join(PROJECT_PATH, "data")

# Make sure 'data' folder exists
os.makedirs(OUTPUTS_PATH, exist_ok=True)

print("Project Path:", PROJECT_PATH)
print("Data Path:", OUTPUTS_PATH)

In [ ]:
%cd $PROJECT_PATH

# Load dataset

In [ ]:
# loading the training set and test set
columns_name=['user_id','item_id','rating','timestamp']
train_data = pd.read_csv(f'{PROJECT_PATH}/training.txt', sep='\t', names=columns_name)
test_data = pd.read_csv(f'{PROJECT_PATH}/test.txt', sep='\t', names=columns_name)

# We take 20% of the original training set as validation
train_data, val_data = train_test_split(
    train_data,
    test_size=0.2,         # 20% validation
    random_state=10,       # ensure reproducibility
    shuffle=True           # shuffle before splitting
)

print(f'The training data:')
display(train_data[['user_id','item_id','rating']].head())
print(f'The shape of the training data: {train_data.shape}')
print('--------------------------------')
print(f'The validation data:')
display(val_data[['user_id','item_id','rating']].head())
print(f'The shape of the validation data: {val_data.shape}')
print('--------------------------------')
print(f'The test data:')
display(test_data[['user_id','item_id','rating']].head())
print(f'The shape of the test data: {test_data.shape}')

In [ ]:
movies = pd.read_csv(f'{PROJECT_PATH}/movies.txt',names=['item_id','title','genres','description'],sep='\t')
movies.head()

In [ ]:
num_users = train_data['user_id'].max()
num_items = train_data['item_id'].max()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Number of users: {num_users}, Number of items: {num_items}, Using device: {device}')

# Helper Functions

In [ ]:
# Create embeddings of the movie items
def create_bert_embeddings(content):
    """
    Generate BERT embeddings for movie content.

    Args:
        content: Content of items

    Returns:
        numpy.ndarray: BERT embeddings matrix
    """
    if not BERT_AVAILABLE:
        print("BERT libraries not available. Install with: pip install transformers torch")
        return None

    if content is None:
        return None

    if isinstance(content, pd.Series):
        content = content.fillna("").astype(str).tolist()
    elif isinstance(content, np.ndarray):
        content = content.astype(str).tolist()

    model_name = 'distilbert-base-uncased'

    print(f"Loading BERT model: {model_name}")

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    # Set device (GPU if available)
    # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using: {device}")
    model.to(device)
    model.eval()

    print(f"Using device: {device}")

    # Generate embeddings in batches
    batch_size = 32  # Adjust based on available memory
    emb = []

    for i in range(0, len(content), batch_size):
        if i % (batch_size * 10) == 0:
            print(f"Processing batch {i//batch_size + 1}/{len(content)//batch_size + 1}")

        batch_texts = content[i:i + batch_size]

        # Tokenize batch
        inputs = tokenizer(
            batch_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )

        # Move to device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate embeddings
        with torch.no_grad():
            outputs = model(**inputs)

            # Use [CLS] token embedding (first token)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            emb.extend(batch_embeddings)

    emb = np.array(emb)

    print(f"BERT embeddings generated: {emb.shape}")
    print(f"Embedding dimension: {emb.shape[1]}")

    return emb

## Get the item embeddings and then save them to local folder



In [ ]:
# Three diffrent content representation (title + genre, description, title + genre + description)

import os
# Create a subfolder for saved outputs to avoid performing expensive computations multiple times
os.makedirs(OUTPUTS_PATH, exist_ok=True)

# Define the file paths for each of the three embedding arrays
path_titlegenres = os.path.join(OUTPUTS_PATH, "item_emb_titlegenres.npy")
path_description = os.path.join(OUTPUTS_PATH, "item_emb_description.npy")
path_full = os.path.join(OUTPUTS_PATH, "item_emb_full.npy")

if os.path.exists(path_titlegenres):
    print(f"Loading Title+Genres embeddings from: {path_titlegenres} 💾")
    item_emb_titlegenres = np.load(path_titlegenres)
    print("✅ Loaded successfully!")
else:
    print("Computing Title+Genres embeddings... ⏳")
    title_genres = movies["title"] + ' ' + movies["genres"]
    item_emb_titlegenres = create_bert_embeddings(title_genres)
    np.save(path_titlegenres, item_emb_titlegenres)
    print(f"✅ Computed and saved to shared drive for future use!")

if os.path.exists(path_description):
    print(f"\nLoading Description embeddings from: {path_description} 💾")
    item_emb_description = np.load(path_description)
    print("✅ Loaded successfully!")
else:
    print("\nComputing Description embeddings... ⏳")
    item_emb_description = create_bert_embeddings(movies["description"])
    np.save(path_description, item_emb_description)
    print(f"✅ Computed and saved to shared drive for future use!")

if os.path.exists(path_full):
    print(f"\nLoading Full Content embeddings from: {path_full} 💾")
    item_emb_full = np.load(path_full)
    print("✅ Loaded successfully!")
else:
    print("\nComputing Full Content embeddings... ⏳")
    title_genres_description = movies["title"] + ' ' + movies["genres"] + ' ' + movies["description"]
    item_emb_full = create_bert_embeddings(title_genres_description)
    np.save(path_full, item_emb_full)
    print(f"✅ Computed and saved to shared drive for future use!")

print("\n--- Final Embedding Shapes ---")
print(f"Title+Genres: {item_emb_titlegenres.shape}")
print(f"Description:  {item_emb_description.shape}")
print(f"Full Content: {item_emb_full.shape}")

In [ ]:
# Get item embedding based on item ID (item id starts at 1)
def get_item_emb(item_id, content_type):
    emb = None

    if content_type == "title_genres":
        emb = item_emb_titlegenres[item_id - 1]
    elif content_type == "description":
        emb = item_emb_description[item_id - 1]
    elif content_type == "full":
        emb = item_emb_full[item_id - 1]

    return emb

In [ ]:
# Given content type (title+genres, description, or title+genres+description) returns the embeddings and ratings of interacted items by user_id
def get_interacted_items_embs_rating(train_data, user_id, content_type):
    embs, ratings = [], []

    # use train_data to retrieve the item_ids that target user interacted
    items_and_ratings = train_data[train_data["user_id"] == user_id][['item_id', 'rating']]
    item_ids = items_and_ratings['item_id']
    ratings = items_and_ratings['rating'].tolist()

    indices = [i - 1 for i in item_ids]

    if content_type == "title_genres":
        embs = item_emb_titlegenres[indices]
    elif content_type == "description":
        embs = item_emb_description[indices]
    elif content_type == "full":
        embs = item_emb_full[indices]

    return embs, ratings

# given content type (title+genres, description, or title+genres+description) and aggregation method (avg, weighted_avg, avg_pos) returns the representation of a user.
def get_user_emb(train_data, user_id, content_type, aggregation_method):

    emb = []

    # obtain item embeddings and corresponding ratings of the items that the user has interacted with
    item_embs, ratings = get_interacted_items_embs_rating(
        train_data, user_id, content_type
    )

    interacted_ratings = np.array(ratings)

    if aggregation_method == 'avg':
        emb = np.mean(item_embs, axis=0)

    elif aggregation_method == 'weighted_avg':
        emb = np.average(item_embs, axis=0, weights=interacted_ratings)

    elif aggregation_method == 'avg_pos':
        ratings = np.array(ratings)

        # Filter for only positively rated items above 4
        positive_mask = ratings >= 4
        positive_embs = item_embs[positive_mask]

        if len(positive_embs) > 0:
            emb = np.mean(positive_embs, axis=0)
        else:
            emb = np.zeros(item_embs.shape[1])

    return emb

In [ ]:
def RMSE(actual_rating, pred_rating):
    # Implement a function that computes RMSE error between actual ratings and predicted ratings.
    # Note that actual_ratings and pred_ratings are lists.

    result = 0.0

    ############# Your code here ############
    result = np.mean((np.array(pred_rating) - np.array(actual_rating))**2)**(1/2)
    #########################################

    return result

# Task 1) Implementation of different recommendation models as well as a hybrid model combining those recommendation models

In [ ]:
# device = 'cuda' if torch.cuda.is_available() else 'mps'
print(f'Using device: {device}')

## 1.1 Content Based Recommendation model


In [ ]:
# compute the predicted rating for a user-item pair
def get_user_item_prediction_content_based(train_data, user_id, item_id, content_type, aggregation_method):
    pred_rating = 0.0

    user_embedding = get_user_emb(train_data, user_id, content_type, aggregation_method)
    item_embedding = get_item_emb(item_id, content_type)

    pred_rating = np.dot(user_embedding, item_embedding)

    return pred_rating

In [ ]:
user_id, item_id = 100, 266
content_type, aggregation_method = 'full', 'avg' # alternatives are content_type={title_genres,description,full} and aggregation_method={avg,weighted_avg,avg_pos}
print('Predicted score for user_id=100 and item_id=266 for content type '+content_type+' and aggregation method '+aggregation_method+':')
print(get_user_item_prediction_content_based(train_data, user_id, item_id, content_type, aggregation_method))

### Function that generates top-k recommendation list for a target user with user-based recommendataion

In [ ]:
def recommend_topk_content_based(train_data, target_user, k, content_type, aggregation_method):
    """
    Generate Top-K recommendations for a target user using Content-based CF.

    Args:
        train_data (pd.DataFrame): ratings data with columns [user_id, item_id, rating]
        target_user (int): user_id of the target user
        k (int): number of items to recommend

    Returns:
        list of (item_id, predicted_score) sorted by score desc
    """
    result = []

    user_rated_items = set(train_data[train_data["user_id"] == target_user]["item_id"])
    all_items = set(train_data["item_id"].unique())
    candidate_items = all_items - user_rated_items

    predictions = []
    for item_id in candidate_items:
        pred_rating = get_user_item_prediction_content_based(
            train_data,
            target_user,
            item_id,
            content_type,
            aggregation_method
        )
        if not np.isnan(pred_rating):  # only keep valid predictions
            predictions.append((item_id, pred_rating))

    ratings = [r for _, r in predictions]
    min_rating, max_rating = min(ratings), max(ratings)

    # print(f"Min predicted rating: {min_rating}, Max predicted rating: {max_rating}")
    # print(predictions[:5])

    if max_rating == min_rating:
        mapped_ratings = [(3.0, i) for i, _ in predictions]
    else:
        mapped_ratings = [
            (
                i,
                1 + (r - min_rating) * 4 / (max_rating - min_rating),
            )
            for i, r in predictions
        ]

    result = sorted(mapped_ratings, key=lambda x: x[1], reverse=True)[:k]

    return result


target_user, k = 1, 50
recommendations = recommend_topk_content_based(train_data, target_user, k, content_type="full", aggregation_method="weighted_avg")
print(
    f"Top-{k} recommendations for user {target_user} with content-based recommendation:"
)
for item, score in recommendations:
    print(f"Item {item}: {score:.4f}")

### Computing RMSE of the content-based recommendation model


In [ ]:
def evaluate_rating_prediction_content_based(train_data, test_data, content_type, aggregation_method):
    # Create a mapping from user_id to user embedding
    users_emb = {}
    users = list(train_data['user_id'].unique())
    for user in tqdm(users, desc="Creating user embedding mapping"):
        users_emb[user] = get_user_emb(train_data, user, content_type, aggregation_method)


    # I do the same for items
    items = list(train_data['item_id'].unique())
    items_emb = {}
    for item in tqdm(items, desc="Creating item embedding mapping"):
        items_emb[item] = get_item_emb(item, content_type)

    actual_ratings, pred_ratings = [], []

    global_mean_rating = train_data['rating'].mean()

    for index, test_row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing test data"):
        user_id = test_row["user_id"]
        item_id = test_row["item_id"]

        user_embedding = users_emb.get(user_id)
        item_embedding = items_emb.get(item_id)

        if user_embedding is None or item_embedding is None:
            # Impute the global average rating if we can't predict
            pred_rating = global_mean_rating
        else:
            # Only calculate dot product if we have both embeddings
            pred_rating = np.dot(user_embedding, item_embedding)

        actual_ratings.append(test_row['rating'])
        pred_ratings.append(pred_rating)

    # Given predicted ratings, map them into [1,5] interval using -> 1 + (pred - min_val) * (4 / (max_val - min_val))

    min_rating = min(pred_ratings)
    max_rating = max(pred_ratings)

    mapped_ratings = [1 + (rating - min_rating) * 4 / (max_rating - min_rating) for rating in pred_ratings]


    mae_value, mse_value, rmse_value = 0.0, 0.0, 0.0


    # compute the metrics: MAE, MSE, RMSE
    # mae_value = MAE(actual_ratings, mapped_ratings)
    # mse_value = MSE(actual_ratings, mapped_ratings)
    rmse_value = RMSE(actual_ratings, mapped_ratings)

    return rmse_value

### Use the RMSE loss to find out which combination of content representation and aggregation method works the best for content-recommendation


In [ ]:
def hyperparameter_search_content_based(train_data, val_data, content_types, aggregation_methods):
    """
    Perform hyperparameter tuning for content based recommendation. It finds the best combination
    of content representations and aggregation methods

    Args:
        train_data (pd.DataFrame): training set with columns ['user_id', 'item_id', 'rating']
        val_data (pd.DataFrame): validation set with columns ['user_id', 'item_id', 'rating']
        content_types: content types list (title_genres, description, full)
        aggregation_methods: avg, weighted_avg, avg_pos

    Returns:
        dict: {'best_config': (best_content_type, best_aggregation_method), 'best_rmse': float, 'results': list of tuples (k, rmse)}
    """

    print("\n=== Content Based Recommendation Hyperparameter Search ===")
    results = []

    for content_type in content_types:
        for agg_method in aggregation_methods:
          print(f"starting with {content_type} and {agg_method}")
          rmse = evaluate_rating_prediction_content_based(
              train_data,
              val_data,
              content_type,
              agg_method
          )

          results.append({
              'content_type': content_type,
              'aggregation_method': agg_method,
              'RMSE': rmse
          })
          print(f"Config: ({content_type}, {agg_method}) -> RMSE = {rmse:.4f}")

    # Get the best performing combinations as the result of this hyperparameter tuning
    best_result = min(results, key=lambda x: x['RMSE'])
    best_config = (best_result['content_type'], best_result['aggregation_method'])
    best_rmse = best_result['RMSE']

    print(f"Best result was achieved by the config: {best_config}")

    return {
        'best_config': best_config,
        'best_rmse': best_rmse,
        'results': results
    }

In [ ]:
import pickle

# Define parameter space
content_types_grid = ['title_genres', 'description', 'full']
aggregation_methods_grid = ['avg', 'weighted_avg', 'avg_pos']

path_content_based_tuning = os.path.join(OUTPUTS_PATH, 'content_based_tuning_results.pkl')

# Run the search or load results
if os.path.exists(path_content_based_tuning):
    print(f"Loading Content-based tuning results from: {path_content_based_tuning} 💾")
    with open(path_content_based_tuning, 'rb') as f:
        content_based_tuning = pickle.load(f)
    print("✅ Loaded successfully!")
else:
    print("Running Content-based hyperparameter search... ⏳")

    content_based_tuning = hyperparameter_search_content_based(
        train_data,
        val_data,
        content_types_grid,
        aggregation_methods_grid
    )

    # Save the results
    with open(path_content_based_tuning, 'wb') as f:
        pickle.dump(content_based_tuning, f)
    print(f"✅ Saved Content-based tuning results to {path_content_based_tuning}")

print("\n--- Content-Based Tuning Results ---")
print(f"Best Config: {content_based_tuning['best_config']}")
print(f"Best Validation RMSE: {content_based_tuning['best_rmse']:.4f}")

rmse_content_based = content_based_tuning['best_rmse']

In [ ]:
# Visualize Content based recommendation hyperparameter tuning results

results_df = pd.DataFrame(content_based_tuning['results'], columns=['content_type', "aggregation_method", 'RMSE'])
results_df["cobination"] = results_df["content_type"] + " + " + results_df["aggregation_method"]
results_df = results_df.sort_values('RMSE')

plt.figure(figsize=(10, 6))
plt.bar(results_df['cobination'], results_df['RMSE'], color='skyblue', edgecolor='black')

plt.title("Content-based CF Hyperparameter Tuning\n(RMSE by content_type + aggregation_method)", fontsize=12)
plt.xlabel("content_type + aggregation_method combination", fontsize=10)
plt.ylabel("Validation RMSE", fontsize=10)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()


## 1.2 User-based neighborhood method (UserKNN)

The **Pearson correlation coefficient** between two users \(x\) and \(y\) is defined as:

$$
r_{xy} = \frac{\sum_{i \in I_{xy}} (x_i - \bar{x})(y_i - \bar{y})}
              {\sqrt{\sum_{i \in I_{xy}} (x_i - \bar{x})^2} \cdot \sqrt{\sum_{i \in I_{xy}} (y_i - \bar{y})^2}}
$$

**Where:**

- $I_{xy}$ = set of items rated by both users  
- $x_i$, $y_i$ = ratings of users \(x\) and \(y\) on item \(i\)  
- $\bar{x}$, $\bar{y}$ = mean ratings of users \(x\) and \(y\) on the common items  



In [ ]:
# define pearson correlation to obtain users similarity
def pearson_correlation(user1_ratings: pd.Series, user2_ratings: pd.Series) -> float:
    # only consider items that have been rated by both users
    common_items = user1_ratings.dropna().index.intersection(user2_ratings.dropna().index)

    if len(common_items) < 2:
        return 0

    u1_ratings = user1_ratings.loc[common_items]
    u2_ratings = user2_ratings.loc[common_items]

    u1_mean = u1_ratings.mean()
    u2_mean = u2_ratings.mean()

    numerator = ((u1_ratings - u1_mean) * (u2_ratings - u2_mean)).sum()
    denominator = np.sqrt(((u1_ratings - u1_mean) ** 2).sum()) * np.sqrt(((u2_ratings - u2_mean) ** 2).sum())

    if denominator == 0:
        return 0.0

    result = numerator / denominator

    if not np.isfinite(result):
        return 0.0

    return result

# compute similarity matrix of size len(users), len(users).
def compute_user_similarity_matrix(train_data: pd.DataFrame) -> pd.DataFrame:
    users = train_data['user_id'].unique()
    user_similarity_matrix = pd.DataFrame(np.zeros((len(users), len(users))), index=users, columns=users)

    for i, usera in enumerate(users):
        for j, userb in enumerate(users):
            if i > j:
                continue
            if usera == userb:
                user_similarity_matrix.loc[usera, userb] = 1
            else:
                sim = pearson_correlation(train_data[train_data["user_id"] == usera].set_index("item_id")["rating"], train_data[train_data["user_id"] == userb].set_index("item_id")["rating"])
                user_similarity_matrix.loc[usera, userb] = sim
                user_similarity_matrix.loc[userb, usera] = sim

    return user_similarity_matrix

# Get top-k most similar users to the target user and return List of tuples: [(neighbor_user_id, similarity), ...] sorted by similarity descending
def get_k_user_neighbors(train_data, user_similarity_matrix: pd.DataFrame, target_user, target_item, k=5):
    top_k_neighbors = []

    # we want to select top k neighbours who have rated the target item
    rated_users = train_data.loc[train_data['item_id'] == target_item, 'user_id'].unique()

    user_similarities = user_similarity_matrix.loc[target_user].drop(target_user)
    # Only keep users who have rated the target item
    user_similarities = user_similarities.loc[user_similarities.index.isin(rated_users)]
    top_k_neighbors = list(user_similarities.nlargest(k).items())

    return top_k_neighbors

In [ ]:
path_user_similarity_matrix = os.path.join(OUTPUTS_PATH, "user_similarity_matrix.pkl")

if os.path.exists(path_user_similarity_matrix):
    print(f"Loading user similarity matrix from: {path_user_similarity_matrix} 💾")
    user_similarity_matrix = pd.read_pickle(path_user_similarity_matrix)
    print("✅ Loaded successfully!")
else:
    print("Computing user similarity matrix... ⏳")
    user_similarity_matrix = compute_user_similarity_matrix(train_data)
    user_similarity_matrix.to_pickle(path_user_similarity_matrix)
    print(f"✅ Computed and saved to shared drive for future use!")

# Verify the index
print(f"User similarity matrix shape: {user_similarity_matrix.shape}")
print(f"Is user 943 in user_similarity_matrix index after loading/computing: {943 in user_similarity_matrix.index}")

In [ ]:
def predict_rating_user_based(
    train_data: pd.DataFrame,
    user_similarity_matrix: pd.DataFrame,
    target_user,
    target_item,
    global_mean_rating, k=5,
):
    """
    Predict rating for target_user and target_item using mean-centered user-based CF.

    Parameters:
    - ratings: pd.DataFrame with columns ['user_id', 'item_id', 'rating']
    - user_similarity_matrix: pd.DataFrame of user-user similarities
    - target_user: user ID
    - target_item: item ID
    - global_mean_rating: global mean rating to use as fallback
    - k: number of neighbors to consider

    Returns:
    - float: predicted rating, or np.nan if not possible
    """
    result = 0.0

    ############# Your code here ############
    user_ratings = train_data[train_data["user_id"] == target_user].set_index("item_id")["rating"]
    item_ratings = train_data[train_data["item_id"] == target_item].set_index("user_id")["rating"]

    user_mean = user_ratings.mean()

    # if user has no rating, fall back to global mean rating
    if pd.isna(user_mean):
        user_mean = global_mean_rating

    # gets top k neighbors who rated the target item
    top_k_neighbors = get_k_user_neighbors(train_data, user_similarity_matrix, target_user, target_item, k)

    # If no neighbors, return user mean
    if not top_k_neighbors:
        return user_mean

    neighbor_ids = [n_id for n_id, similarity in top_k_neighbors]
    neighbour_id_to_mean_rating = train_data[train_data["user_id"].isin(neighbor_ids)].groupby('user_id')['rating'].mean()
    # # global mean for neighbors with no ratings
    # neighbour_id_to_mean_rating = neighbour_id_to_mean_rating.fillna(global_mean_rating)

    numerator = 0.0
    denominator = 0.0

    for neighbour_id, similarity in top_k_neighbors:
        if neighbour_id in item_ratings.index:
            numerator += similarity * (item_ratings.loc[neighbour_id] - neighbour_id_to_mean_rating[neighbour_id])
            denominator += abs(similarity)

    if denominator == 0:
        return user_mean 

    result = user_mean + (numerator/denominator)

    result = np.clip(result, 1, 5)

    #########################################

    return result

In [ ]:
target_user, target_item, k = 1, 17, 50
global_mean_rating = train_data["rating"].mean()
print(
    f"The actual rating for user {target_user} and item {target_item} is 3. The predicted rating by user-based CF for user {target_user} and item {target_item} is {predict_rating_user_based(train_data, user_similarity_matrix, target_user, target_item, global_mean_rating, k):.4f}"
)

### Function that generates top-k recommendation list for a target user with user-based recommendataion

In [ ]:
def recommend_topk_user_based(train_data, user_similarity_matrix, target_user, k_user, k=5):
    """
    Generate Top-K recommendations for a target user using User-based CF.

    Args:
        train_data (pd.DataFrame): ratings data with columns [user_id, item_id, rating]
        user_similarity_matrix (pd.DataFrame): precomputed user-user similarity matrix
        target_user (int): user_id of the target user
        k (int): number of items to recommend

    Returns:
        list of (item_id, predicted_score) sorted by score desc
    """
    result = []

    user_rated_items = set(train_data[train_data["user_id"] == target_user]["item_id"])
    all_items = set(train_data["item_id"].unique())
    candidate_items = all_items - user_rated_items

    predictions = []
    for item_id in candidate_items:
        pred_rating = predict_rating_user_based(
            train_data,
            user_similarity_matrix,
            target_user,
            item_id,
            global_mean_rating,
            k_user,
        )
        if not np.isnan(pred_rating):  # only keep valid predictions
            predictions.append((item_id, pred_rating))

    result = sorted(predictions, key=lambda x: x[1], reverse=True)[:k]

    return result


target_user, k_user, k = 1, 50, 50
recommendations = recommend_topk_user_based(
    train_data, user_similarity_matrix, target_user, k_user, k
)
print(f"Top-{k} recommendations for user {target_user} with user-based recommendation:")
for item, score in recommendations:
    print(f"Item {item}: {score:.4f}")

### Computing RMSE of the User based KNN recommendation model



In [ ]:
def evaluate_rating_prediction_user_based(train_data, test_data, user_similarity_matrix, k):
    actual_ratings, pred_ratings = [], []
    global_mean_rating = train_data["rating"].mean()


    for index, test_row in tqdm(test_data.iterrows(), total=len(test_data), desc="Processing test data"):
        user_id = test_row["user_id"]
        item_id = test_row["item_id"]

        pred_rating = predict_rating_user_based(train_data, user_similarity_matrix, user_id, item_id, global_mean_rating, k)

        if not np.isnan(pred_rating):
            actual_ratings.append(test_row['rating'])
            pred_ratings.append(pred_rating)

    # Given predicted ratings, map them into [1,5] interval using -> 1 + (pred - min_val) * (4 / (max_val - min_val))

    min_rating = min(pred_ratings)
    max_rating = max(pred_ratings)

    if max_rating == min_rating:
        min_rating = max_rating - 0.001

    mapped_ratings = [1 + (rating - min_rating) * 4 / (max_rating - min_rating) for rating in pred_ratings]

    mae_value, mse_value, rmse_value = 0.0, 0.0, 0.0

    # compute the metrics: MAE, MSE, RMSE
    # mae_value = MAE(actual_ratings, mapped_ratings)
    # mse_value = MSE(actual_ratings, mapped_ratings)
    rmse_value = RMSE(actual_ratings, mapped_ratings)

    return rmse_value

### Use the RMSE loss to find out which combination of content representation and aggregation method works the best for content-recommendation




In [ ]:
def hyperparameter_search_user_based(train_data, val_data, user_similarity_matrix, ks):
    """
    Perform hyperparameter tuning for user KNN based recommendation. It finds the best k value

    Args:
        train_data (pd.DataFrame): training set with columns ['user_id', 'item_id', 'rating']
        val_data (pd.DataFrame): validation set with columns ['user_id', 'item_id', 'rating']
        user_similarity_matrix: user similarity matrix
        ks: list of ks to test out

    Returns:
            dict: {'best_k': int, 'best_rmse': float, 'results': list of tuples (k, rmse)}
    """

    print("\n=== User Based Recommendation Hyperparameter Search ===")
    results = []

    for k in ks:
          print(f"Testing k value={k}")
          rmse = evaluate_rating_prediction_user_based(train_data, val_data, user_similarity_matrix, k)

          results.append({
              'k': k,
              'RMSE': rmse,
          })
          print(f"Config: k={k} -> RMSE = {rmse:.4f}")

    # Get the best performing combinations as the result of this hyperparameter tuning
    best_result = min(results, key=lambda x: x['RMSE'])
    best_k = best_result['k']
    best_rmse = best_result['RMSE']
    print(f"\n✅ Best UserKNN config: k={best_k}, RMSE={best_rmse:.4f}")

    return {'best_k': best_k, 'best_rmse': best_rmse, 'results': results}

In [ ]:
path_user_knn_tuning = os.path.join(OUTPUTS_PATH, "user_knn_tuning_result.pkl")
ks = [5, 10, 20, 30, 50, 70, 100]

user_knn_tuning = None

if os.path.exists(path_user_knn_tuning):
    print(f"Loading UserKNN tuning results from: {path_user_knn_tuning} 💾")
    with open(path_user_knn_tuning, 'rb') as f:
        user_knn_tuning = pickle.load(f)
    print("✅ Loaded successfully!")
else:
    print("Running UserKNN hyperparameter search... ⏳")
    user_knn_tuning = hyperparameter_search_user_based(train_data, val_data, user_similarity_matrix, ks)
    with open(path_user_knn_tuning, 'wb') as f:
        pickle.dump(user_knn_tuning, f)
    print(f"✅ Saved UserKNN tuning results to {path_user_knn_tuning}")

In [ ]:
# Visualize UserKNN hyperparameter tuning results

results_df = pd.DataFrame(user_knn_tuning['results'], columns=['k', 'RMSE'])

plt.figure(figsize=(8,5))
plt.plot(results_df['k'], results_df['RMSE'], marker='o', linestyle='-', linewidth=2)
plt.title("User-based CF Hyperparameter Tuning (RMSE vs. k)")
plt.xlabel("Number of Neighbours (k)")
plt.ylabel("Validation RMSE")
plt.grid(True, linestyle='--', alpha=0.6)

# Highlight the best k
best_k = user_knn_tuning['best_k']
best_rmse = user_knn_tuning['best_rmse']
rmse_user_based = best_rmse
plt.scatter(best_k, best_rmse, color='red', s=100, label=f"Best k={best_k} (RMSE={best_rmse:.4f})")
plt.legend()
plt.show()

## 1.3 Item-based neighborhood method (ItemKNN)

The **cosine similarity** between two items $i$ and $j$ is defined as:

$$
\text{sim}(i,j) = \frac{\sum_{u \in U_{ij}} r_{u,i} \cdot r_{u,j}}
                      {\sqrt{\sum_{u \in U_{ij}} r_{u,i}^2} \cdot \sqrt{\sum_{u \in U_{ij}} r_{u,j}^2}}
$$

Where:

- $r_{u,i}$ = rating of user $u$ on item $i$
- $r_{u,j}$ = rating of user $u$ on item $j$
- $U_{ij}$ = set of users who have rated both items $i$ and $j$


In [ ]:
def cosine_similarity(item1_ratings: pd.Series, item2_ratings: pd.Series) -> float:
    """
    Compute cosine similarity between two items' rating vectors.
    Only common users are considered.

    Parameters:
    - item1_ratings, item2_ratings: pd.Series indexed by user_id

    Returns:
    - float: cosine similarity between -1 and 1
    """
    result = 0.0

    common_users = item1_ratings.index.intersection(item2_ratings.index)
    if len(common_users) == 0:
        return 0.0  # no common users, similarity is 0

    x = item1_ratings[common_users].values
    y = item2_ratings[common_users].values

    numerator = np.dot(x, y)
    denominator = np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y))
    if denominator == 0:
        return 0.0  # avoid division by zero

    result = numerator / denominator

    return result

item1, item2 = 1, 2
item1_ratings = train_data[train_data['item_id'] == item1].set_index('user_id')['rating']
item2_ratings = train_data[train_data['item_id'] == item2].set_index('user_id')['rating']
print(f"Cosine similarity between items {item1} and {item2} is {cosine_similarity(item1_ratings, item2_ratings):.4f}")

### Item-Item Similarity Matrix

In [ ]:
def compute_item_similarity_matrix(train_data: pd.DataFrame) -> pd.DataFrame:
    """
    Compute item-item similarity matrix using cosine similarity.

    Parameters:
    - ratings: pd.DataFrame with columns ['user_id', 'item_id', 'rating']

    Returns:
    - pd.DataFrame: item-item similarity matrix (rows & cols = item_ids)
    """
    items = train_data['item_id'].unique()
    item_similarity_matrix = pd.DataFrame(np.zeros((len(items), len(items))), index=items, columns=items)

    for i, i1 in enumerate(items):
        i1_ratings = train_data[train_data['item_id'] == i1].set_index('user_id')['rating']
        for j, i2 in enumerate(items):
            if j > i:  # Only compute for the upper triangle
                i2_ratings = train_data[train_data['item_id'] == i2].set_index('user_id')['rating']
                sim = cosine_similarity(i1_ratings, i2_ratings)
                item_similarity_matrix.at[i1, i2] = sim
                item_similarity_matrix.at[i2, i1] = sim  # Mirror the value
            elif i == j:
                item_similarity_matrix.at[i1, i2] = 1.0  # Similarity with self is 1

    return item_similarity_matrix

# start_time = time.time()
# item_similarity_matrix = compute_item_similarity_matrix(train_data)
# end_time = time.time()
# print(f'Running time: {end_time - start_time:.4f} seconds')

In [ ]:
path_item_similarity_matrix = os.path.join(OUTPUTS_PATH, "item_similarity_matrix.pkl")

if os.path.exists(path_item_similarity_matrix):
    print(f"Loading item similarity matrix from: {path_item_similarity_matrix} 💾")
    item_similarity_matrix = pd.read_pickle(path_item_similarity_matrix)

    # ensure IDs are int
    item_similarity_matrix.index = item_similarity_matrix.index.astype(int)
    item_similarity_matrix.columns = item_similarity_matrix.columns.astype(int)

    print(f"✅ Loaded successfully with shape {item_similarity_matrix.shape} and item IDs preserved!")
else:
    print("Computing item similarity matrix... ⏳")
    start_time = time.time()
    item_similarity_matrix = compute_item_similarity_matrix(train_data)
    end_time = time.time()

    # ensure IDs are int
    item_similarity_matrix.index = item_similarity_matrix.index.astype(int)
    item_similarity_matrix.columns = item_similarity_matrix.columns.astype(int)

    # save properly with labels
    item_similarity_matrix.to_pickle(path_item_similarity_matrix)
    print(f"✅ Computed and saved to {path_item_similarity_matrix} (took {end_time - start_time:.2f}s)")

In [ ]:
print("Matrix shape:", item_similarity_matrix.shape)
print("Index dtype:", item_similarity_matrix.index.dtype)
print("Column dtype:", item_similarity_matrix.columns.dtype)
print("Sample item IDs:", item_similarity_matrix.index[:10].tolist())
print("Accessing item 599 row works?", 599 in item_similarity_matrix.index)


> Instead of calculating the similarity for all pairs, we only compute it for one half of the matrix (upper or lower triangle) and then mirror the values.

### Get k most similar item along with the similarity values to a target item

In [ ]:
def get_k_item_neighbors(item_similarity_matrix: pd.DataFrame, target_item, k=5):
    """
    Retrieve top-k most similar items to the target item.

    Parameters:
    - item_similarity_matrix: pd.DataFrame, item-item similarity
    - target_item: item ID
    - k: number of neighbors

    Returns:
    - List of tuples: [(neighbor_item_id, similarity), ...]
    """
    top_k_neighbors = []

    similarities = item_similarity_matrix.loc[target_item].drop(index=target_item)
    top_k_neighbors = similarities.sort_values(ascending=False).head(k)
    top_k_neighbors = list(zip(top_k_neighbors.index, top_k_neighbors.values))

    return top_k_neighbors

target_item, k = 1, 10
print(f"Neighbors of item {target_item} are:")
get_k_item_neighbors(item_similarity_matrix, target_item, k)

### Function that predicts the rating that a target user might give to a target item using item-item similarity matrix

The **predicted rating** for a target user $u$ on a target item $i$ using item-based collaborative filtering is:

$$
\hat{r}_{u,i} = \frac{\sum_{j \in N(i)} s(i,j) \cdot r_{u,j}}{\sum_{j \in N(i)} |s(i,j)|}
$$

Where:

- $\hat{r}_{u,i}$ = predicted rating of user $u$ on item $i$
- $N(i)$ = set of top-$k$ most similar items to item $i$ that user $u$ has rated
- $s(i,j)$ = similarity between item $i$ and item $j$
- $r_{u,j}$ = rating of user $u$ on item $j$

In [ ]:
def predict_rating_item_based(train_data: pd.DataFrame, item_similarity_matrix: pd.DataFrame, target_user, target_item, k=5):
    """
    Predict rating using item-based CF (non-mean centric).

    Parameters:
    - ratings: pd.DataFrame ['user_id', 'item_id', 'rating']
    - item_similarity_matrix: item-item similarity DataFrame
    - target_user: user ID
    - target_item: item ID
    - k: number of neighbors to use

    Returns:
    - float: predicted rating, or np.nan if not enough data
    """
    result = 0.0

    user_ratings_all = train_data[train_data["user_id"] == target_user]["rating"]
    user_mean = user_ratings_all.mean()

    if target_item not in item_similarity_matrix.index:
        return user_mean

    user_ratings = train_data[train_data['user_id'] == target_user][['item_id', 'rating']]
    rated_items = user_ratings['item_id'].values

    # Get top-k similar items to target_item that user has rated
    similarities = item_similarity_matrix.loc[target_item, rated_items]
    top_k_idx = np.argsort(similarities.values)[::-1][:k]
    top_k_items = rated_items[top_k_idx]
    top_k_sims = similarities.values[top_k_idx]

    numerator = 0.0
    denominator = 0.0
    for item, sim in zip(top_k_items, top_k_sims):
        rating = user_ratings[user_ratings['item_id'] == item]['rating'].values[0]
        numerator += sim * rating
        denominator += abs(sim)

    if denominator == 0:
        # If no similar items, return user's mean rating or np.nan
        return user_mean

    result = numerator / denominator

    return result

target_user, target_item, k = 1, 17, 50
print(f"The actual rating for user {target_user} and item {target_item} is 3. The predicted rating by item-based CF for user {target_user} and item {target_item} is {predict_rating_item_based(train_data, item_similarity_matrix, target_user, target_item, k):.4f}")

### Function that generates top-k recommendation list for a target user with item-based recommendataion

In [ ]:
def recommend_topk_item_based(train_data, item_similarity_matrix, target_user, k_item, k=5):
    """
    Generate Top-K recommendations for a target user using Item-based CF.

    Args:
        train_data (pd.DataFrame): ratings data with columns [user_id, item_id, rating]
        item_similarity_matrix (pd.DataFrame): precomputed item-item similarity matrix
        target_user (int): user_id of the target user
        k (int): number of items to recommend

    Returns:
        list of (item_id, predicted_score) sorted by score desc
    """
    result = []

    user_rated_items = set(train_data[train_data['user_id'] == target_user]['item_id'])
    all_items = set(train_data['item_id'].unique())
    candidate_items = all_items - user_rated_items

    predictions = []
    for item_id in candidate_items:
        pred_rating = predict_rating_item_based(
            train_data, item_similarity_matrix, target_user, item_id, k_item
        )
        if not np.isnan(pred_rating):  # only keep valid predictions
            predictions.append((item_id, pred_rating))

    result = sorted(predictions, key=lambda x: x[1], reverse=True)[:k]

    return result

target_user, k_item, k = 1, 50, 50
recommendations = recommend_topk_item_based(train_data, item_similarity_matrix, target_user, k_item, k)
print(f"Top-{k} recommendations for user {target_user}:")
for item, score in recommendations:
    print(f"Item {item}: {score:.4f}")

### Item based CF Hyperparameter Tuning

> Since this is a rating task, we will be using RMSE to tune the number of neighbours to take into consideration, namely `k`.

In [ ]:
def evaluate_rating_prediction_item_based(train_data, val_data, item_similarity_matrix, k):

    def RMSE(actual_rating, pred_rating):
        return np.mean((np.array(pred_rating) - np.array(actual_rating))**2)**0.5

    preds, actuals = [], []

    for _, row in val_data.iterrows():
        user_id, item_id, true_rating = int(row['user_id']), int(row['item_id']), row['rating']
        pred = predict_rating_item_based(train_data, item_similarity_matrix, user_id, item_id, k=k)

        # Skip if prediction not possible (NaN)
        if not np.isnan(pred):
            preds.append(pred)
            actuals.append(true_rating)

    rmse = RMSE(actuals, preds)
    return rmse

def hyperparameter_search_item_based(train_data, val_data, item_similarity_matrix, k_values=[5, 10, 20, 30, 50, 70, 100]):
    """
    Perform hyperparameter tuning for Item-based CF using validation RMSE.
    Only searches over k (number of neighbors), but can be extended easily.

    Args:
        train_data (pd.DataFrame): training set with columns ['user_id', 'item_id', 'rating']
        val_data (pd.DataFrame): validation set with columns ['user_id', 'item_id', 'rating']
        item_similarity_matrix (pd.DataFrame): precomputed item-item similarity matrix
        k_values (list): list of k values to test

    Returns:
        dict: {'best_k': int, 'best_rmse': float, 'results': list of tuples (k, rmse)}
    """

    print("\n=== Item-based CF Hyperparameter Search ===")
    results = []

    for k in k_values:
        print(f"\nEvaluating k = {k}...")
        rmse = evaluate_rating_prediction_item_based(train_data, val_data, item_similarity_matrix, k)

        results.append({
            'k': k,
            'RMSE': rmse,
        })

        if np.isnan(rmse):
            print(f"k={k}: Evaluation failed (no valid predictions).")
        else:
            print(f"k={k}: Validation RMSE = {rmse:.4f}")

    valid_results = [r for r in results if not np.isnan(r['RMSE'])]

    if not valid_results:
        print("Hyperparameter search failed to produce any valid results.")
        return {'best_k': None, 'best_rmse': np.nan, 'results': results}

    best_result = min(valid_results, key=lambda x: x['RMSE'])
    best_k = best_result['k']
    best_rmse = best_result['RMSE']

    print(f"\n✅ Best ItemKNN config: k={best_k}, RMSE={best_rmse:.4f}")

    return {'best_k': best_k, 'best_rmse': best_rmse, 'results': results}


In [ ]:
import pickle

path_item_knn_tuning = os.path.join(OUTPUTS_PATH, 'item_knn_tuning_results.pkl')


if os.path.exists(path_item_knn_tuning):
    print(f"Loading ItemKNN tuning results from: {path_item_knn_tuning} 💾")
    with open(path_item_knn_tuning, 'rb') as f:
        item_knn_tuning = pickle.load(f)
    print("✅ Loaded successfully!")
else:
    print("Running ItemKNN hyperparameter search... ⏳")
    item_knn_tuning = hyperparameter_search_item_based(train_data, val_data, item_similarity_matrix)
    with open(path_item_knn_tuning, 'wb') as f:
        pickle.dump(item_knn_tuning, f)
    print(f"✅ Saved ItemKNN tuning results to {path_item_knn_tuning}")

In [ ]:
best_k = item_knn_tuning['best_k']
# takes 20 minutes to run

In [ ]:
# Visualize ItemKNN hyperparameter tuning results

results_df = pd.DataFrame(item_knn_tuning['results'], columns=['k', 'RMSE'])

plt.figure(figsize=(8,5))
plt.plot(results_df['k'], results_df['RMSE'], marker='o', linestyle='-', linewidth=2)
plt.title("Item-based CF Hyperparameter Tuning (RMSE vs. k)")
plt.xlabel("Number of Neighbours (k)")
plt.ylabel("Validation RMSE")
plt.grid(True, linestyle='--', alpha=0.6)

# Highlight the best k
best_k = item_knn_tuning['best_k']
best_rmse = item_knn_tuning['best_rmse']
rmse_item_based = best_rmse
plt.scatter(best_k, best_rmse, color='red', s=100, label=f"Best k={best_k} (RMSE={best_rmse:.4f})")
plt.legend()
plt.show()

![image.png](attachment:fa957900-1d21-4a49-8ea6-e0b8aeb2d891.png)

> Based on the results and as seen in the visualization, we take 70 to be the best number of neighbours for item based CF. We have tried both np.nan and user mean as fallback conditions (denominator being zero or the target item not being in the similarity matrix) but they rarely occurred during the evaluation on val_data, so the returned np.nan (and skipped) or returned user_mean (and included its error) do not make a significant difference to the final averaged RMSE. We can verify the low number of such cases by counting them. However, for the sake of the following analysis, we will consider the average as fallback for both UserKNN and ItemKNN.

In [ ]:
# Verify the hypothesis about the number of fallbacks in UserKNN and ItemKNN

print("Verifying the number of fallbacks during validation for CF models...")


# --- UserKNN Fallback Check ---
def count_userknn_fallbacks(train_data, val_data, user_similarity_matrix, k):
    fallbacks = 0
    total_predictions = 0
    global_mean_rating = train_data["rating"].mean()

    # Store user means to avoid recomputing
    user_means = train_data.groupby('user_id')['rating'].mean().fillna(global_mean_rating)

    for index, test_row in tqdm(val_data.iterrows(), total=len(val_data), desc="UserKNN Fallback Check"):
        user_id = test_row["user_id"]
        item_id = test_row["item_id"]
        total_predictions += 1

        # Check for unknown user
        if user_id not in user_similarity_matrix.index or pd.isna(user_means.get(user_id)):
            fallbacks += 1  # Fallback to global mean
            continue

        # Check for neighbours
        rated_users = train_data.loc[train_data['item_id'] == item_id, 'user_id'].unique()
        user_similarities = user_similarity_matrix.loc[user_id].drop(user_id)
        user_similarities_for_item = user_similarities.loc[user_similarities.index.isin(rated_users)]
        top_k_neighbors = user_similarities_for_item.nlargest(k)

        if len(top_k_neighbors) == 0:
            fallbacks += 1  # Fallback to user mean
            continue

        # Check for zero denominator
        denominator = top_k_neighbors.abs().sum()
        if denominator == 0:
            fallbacks += 1  # Fallback to user mean

    return fallbacks, total_predictions


# --- ItemKNN Fallback Check ---
def count_itemknn_fallbacks(train_data, val_data, item_similarity_matrix, k):
    fallbacks = 0
    total_predictions = 0

    # Store user means to avoid recomputing
    user_means = train_data.groupby('user_id')['rating'].mean()

    for index, test_row in tqdm(val_data.iterrows(), total=len(val_data), desc="ItemKNN Fallback Check"):
        user_id = test_row["user_id"]
        item_id = test_row["item_id"]
        total_predictions += 1

        user_mean = user_means.get(user_id, train_data['rating'].mean())

        # Check for unknown item or unrated user
        if item_id not in item_similarity_matrix.index or pd.isna(user_mean):
            fallbacks += 1  # Fallback to user mean or global mean
            continue

        user_ratings = train_data[train_data['user_id'] == user_id][['item_id', 'rating']]
        rated_items = user_ratings['item_id'].values

        # Check for rated items
        if len(rated_items) == 0:
            fallbacks += 1  # Fallback to user mean (or global)
            continue

        # Get top-k similar items to target_item that user has rated
        similarities = item_similarity_matrix.loc[item_id].reindex(rated_items).dropna()
        top_k_items_sim = similarities.nlargest(k)

        if len(top_k_items_sim) == 0:
            fallbacks += 1  # Fallback to user mean
            continue

        # Check for zero denominator
        denominator = top_k_items_sim.abs().sum()
        if denominator == 0:
            fallbacks += 1  # Fallback to user mean

    return fallbacks, total_predictions


# Use best k from previous tuning (k_user=50, k_item=70)
k_user_best = 50
k_item_best = 70
total_val_data_size = len(val_data)

ub_fallbacks, ub_total = count_userknn_fallbacks(train_data, val_data, user_similarity_matrix, k=k_user_best)
ib_fallbacks, ib_total = count_itemknn_fallbacks(train_data, val_data, item_similarity_matrix, k=k_item_best)

print("\n--- Fallback Verification Results (Validation Set) ---")
print(f"Total Validation Pairs: {total_val_data_size}")
print(f"UserKNN (k={k_user_best}): Fallbacks = {ub_fallbacks} ({ub_fallbacks / ub_total * 100:.2f}%)")
print(f"ItemKNN (k={k_item_best}): Fallbacks = {ib_fallbacks} ({ib_fallbacks / ib_total * 100:.2f}%)")
print("-" * 40)
print("Conclusion: The number of fallbacks is indeed low, supporting the hypothesis.")


In [ ]:
# remember the best value for item k and user k to use later
best_k_user = user_knn_tuning['best_k']
best_k_item = item_knn_tuning['best_k']

## 1.4 Matrix Factorization model (MF)

### MF Model

In [ ]:
class MFModel(nn.Module):
    def __init__(self, num_users, num_items, num_factors=32):
        super(MFModel, self).__init__()
        # User and item embeddings
        self.user_emb = nn.Embedding(num_users, num_factors)
        self.item_emb = nn.Embedding(num_items, num_factors)
        self._init_weights()

    def _init_weights(self):
        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)

    def forward(self, user_ids, item_ids):
        # Dot product between user and item embeddings
        u = self.user_emb(user_ids)
        i = self.item_emb(item_ids)
        pred = (u * i).sum(dim=1)
        return pred

In [ ]:
class MFTrainer:
    def __init__(self, num_users, num_items, num_factors=32, lr=0.01, reg=1e-4, epochs=10, device=device, outputs_path="outputs"):
        self.device = device
        self.num_users = num_users
        self.num_items = num_items
        self.outputs_path = os.path.join(outputs_path, 'MF')
        os.makedirs(self.outputs_path, exist_ok=True)
        self.model_file = os.path.join(self.outputs_path, "mf_model.pt")

        self.model = MFModel(num_users, num_items, num_factors).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=reg)
        self.epochs = epochs
        self.loss_fn = nn.MSELoss()

    def train(self, train_data):
        users = torch.LongTensor(train_data['user_id'].values - 1).to(self.device)
        items = torch.LongTensor(train_data['item_id'].values - 1).to(self.device)
        ratings = torch.FloatTensor(train_data['rating'].values).to(self.device)

        dataset = torch.utils.data.TensorDataset(users, items, ratings)
        loader = torch.utils.data.DataLoader(dataset, batch_size=256, shuffle=True)

        print("Training MF...")
        for epoch in range(self.epochs):
            total_loss = 0
            for u, i, r in loader:
                pred = self.model(u, i)
                loss = self.loss_fn(pred, r)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                total_loss += loss.item()
            print(f"Epoch {epoch+1}/{self.epochs}, Loss={total_loss/len(loader):.4f}")

        # Save trained model
        torch.save(self.model.state_dict(), self.model_file)
        self.trained = True
        print(f"MF model saved to {self.model_file}")

    def predict(self, user_id, item_id):
        self.model.eval()
        user = torch.LongTensor([user_id - 1]).to(self.device)
        item = torch.LongTensor([item_id - 1]).to(self.device)
        with torch.no_grad():
            pred = self.model(user, item)
        return pred.item()

    def evaluate(self, data: pd.DataFrame) -> float:
        self.model.eval()

        users = torch.LongTensor(data["user_id"].values - 1).to(self.device)
        items = torch.LongTensor(data["item_id"].values - 1).to(self.device)
        ratings = torch.FloatTensor(data["rating"].values).to(self.device)

        rmse = 0.0
        with torch.no_grad():
            preds = self.model(users, items)
            rmse = torch.sqrt(torch.mean((preds - ratings) ** 2)).item()

        return rmse

### MF Hyperparam Searching and Training

In [ ]:
def hyperparameter_search_MF(train_data, val_data, num_users, num_items, device, OUTPUTS_PATH, load_best_only=False):
    """
    Perform grid search over MF hyperparameters.
    If load_best_only=True, directly load the previously best model without retraining.
    """

    mf_dir = os.path.join(OUTPUTS_PATH, "MF")
    os.makedirs(mf_dir, exist_ok=True)

    best_config_file = os.path.join(mf_dir, "best_mf_config.json")
    best_model_file = os.path.join(mf_dir, "mf_model.pt")

    # ===============================
    # Case 1: Only load best model
    # ===============================
    if load_best_only:
        if not os.path.exists(best_config_file) or not os.path.exists(best_model_file):
            raise FileNotFoundError("⚠️ No saved best MF model found. Run grid search first.")

        with open(best_config_file, "r") as f:
            best_result = json.load(f)

        print(f"🔹 Loading best MF model: {best_result}")

        best_num_factors = best_result["params"][0]
        best_lr = best_result["params"][1]
        best_reg = best_result["params"][2]

        mf_trainer = MFTrainer(
            num_users, num_items,
            num_factors=best_num_factors, lr=best_lr, reg=best_reg,
            epochs=10, device=device, outputs_path=OUTPUTS_PATH
        )

        mf_trainer.model.load_state_dict(torch.load(best_model_file, map_location=str(device)))
        print(f"✅ Loaded pre-trained MF model from {best_model_file}")

        print("MF Prediction Example:", mf_trainer.predict(1, 50))
        return mf_trainer, best_result["rmse"]

    # ===============================
    # Case 2: Full grid search
    # ===============================
    mf_param_grid = {
        "num_factors": [16, 32, 64],
        "lr": [0.005, 0.01],
        "reg": [1e-4, 1e-3],
    }

    best_result = {"rmse": float("inf"), "params": None}

    print("\n=== MF Hyperparameter Search ===")
    for num_factors, lr, reg in itertools.product(
            mf_param_grid["num_factors"],
            mf_param_grid["lr"],
            mf_param_grid["reg"]):

        print(f"\nTraining MF with num_factors={num_factors}, lr={lr}, reg={reg}")
        mf_trainer = MFTrainer(
            num_users, num_items,
            num_factors=num_factors, lr=lr, reg=reg,
            epochs=5, device=device, outputs_path=OUTPUTS_PATH
        )

        mf_trainer.train(train_data)

        # ---- Evaluate on validation set using RMSE ----
        rmse = mf_trainer.evaluate(val_data)
        print(f"Validation RMSE = {rmse:.4f}")

        if rmse < best_result["rmse"]:
            best_result["rmse"] = rmse
            best_result["params"] = (num_factors, lr, reg)

    print(f"\n✅ Best MF Config: factors={best_result['params'][0]}, "
          f"lr={best_result['params'][1]}, reg={best_result['params'][2]}, "
          f"RMSE={best_result['rmse']:.4f}")

    # Save best configuration to JSON file
    with open(best_config_file, "w") as f:
        json.dump(best_result, f, indent=4)
        print(f"💾 Saved best MF configuration to {best_config_file}")

    # ===============================
    # Train final best model
    # ===============================
    best_num_factors, best_lr, best_reg = best_result["params"]
    final_mf = MFTrainer(
        num_users, num_items,
        num_factors=best_num_factors, lr=best_lr, reg=best_reg,
        epochs=10, device=device, outputs_path=OUTPUTS_PATH
    )
    final_mf.train(train_data)

    # Save the best model as a fixed filename
    torch.save(final_mf.model.state_dict(), best_model_file)
    print(f"💾 Best MF model saved to {best_model_file}")

    print("MF Prediction Example:", final_mf.predict(1, 50))
    return final_mf, best_result["rmse"]

In [ ]:
# mf_model, rmse_mf = hyperparameter_search_MF(train_data, val_data, num_users, num_items, device, OUTPUTS_PATH)
mf_model, rmse_mf = hyperparameter_search_MF(train_data, val_data, num_users, num_items, device, OUTPUTS_PATH, load_best_only=True)  # debug only

In [ ]:
print(f"Minimum Validation RMSE from MF: {rmse_mf:.4f}")

### Recommend Top K items based on matrix factorization

In [ ]:
import torch
import numpy as np
import pandas as pd


def recommend_topk_mf(
    mf_trainer: MFTrainer, train_data: pd.DataFrame, target_user: int, k: int
):
    """
    Generates Top-K recommendations for a target user using the trained MF model.

    This function predicts scores for all items the user has not yet rated
    in a single batch for efficiency.

    Args:
        mf_trainer (MFTrainer): The trained MFTrainer object from your code.
        train_data (pd.DataFrame): Training data used to find already-rated items.
        target_user (int): The target user ID (assumed to be 1-indexed).
        k (int): The number of items to recommend.

    Returns:
        list: A list of k item IDs (1-indexed), sorted by predicted score.
    """
    mf_trainer.model.eval()
    user_rated_items = set(train_data[train_data["user_id"] == target_user]["item_id"])
    all_items = set(range(1, mf_trainer.num_items + 1))
    candidate_items = all_items - user_rated_items
    candidate_items_list = sorted(list(candidate_items))

    # 3. Prepare tensors for batch prediction
    device = mf_trainer.device

    # Create a tensor for the user ID (0-indexed), repeated for all candidate items
    users_tensor = (
        torch.LongTensor([target_user - 1]).expand(len(candidate_items_list)).to(device)
    )

    # Create a tensor for all 0-indexed candidate item IDs
    items_tensor = torch.LongTensor(np.array(candidate_items_list) - 1).to(device)

    # 4. Get all predictions in one batch
    with torch.no_grad():
        predictions = mf_trainer.model(users_tensor, items_tensor)


    scores = predictions.cpu().numpy()
    item_scores = list(zip(candidate_items_list, scores))

    item_scores.sort(key=lambda x: x[1], reverse=True)

    top_k_recommendations = [item_id for item_id, score in item_scores[:k]]

    return top_k_recommendations

In [ ]:
target_user = 1
k = 10

recommendations = recommend_topk_mf(
    mf_trainer=mf_model, train_data=train_data, target_user=target_user, k=k
)

print(f"Top-{k} recommendations based on matrix factorizationfor user {target_user}:")
print(recommendations)

## 1.5 Bayesian Personalized Ranking method (BPR)

BPR is a pairwise ranking optimization method for personalized recommendation. The main idea of BPR is to optimize the ranking of items for each user by maximizing the difference in predicted scores between items that a user has interacted with (positive items) and items that they have not interacted with (negative items).

Here, MF is served as the underlying model for BPR. Objective function of BPR (MF-based) is defined as follows:

$$\sum_{(u, i, j) \in D_S} \ln \sigma(\hat{x}_{ui} - \hat{x}_{uj}) - \lambda_{\Theta} ||\Theta||^2$$

where $D_S$ is the set of all user-item pairs such that user $u$ has interacted with item $i$ (positive item) but not with item $j$ (negative item), $\sigma$ is the sigmoid function, $\hat{x}_{ui}$ and $\hat{x}_{uj}$ are the predicted scores for user $u$ on items $i$ and $j$, respectively, $\Theta$ represents the model parameters (user and item latent factors), and $\lambda_{\Theta}$ is the regularization parameter to prevent overfitting.

### BPR Model

In [ ]:
class BPRModel(nn.Module):
    def __init__(self, num_users, num_items, num_factors=32):
        super(BPRModel, self).__init__()
        # User and item embeddings without bias
        self.user_emb = nn.Embedding(num_users, num_factors)
        self.item_emb = nn.Embedding(num_items, num_factors)
        self._init_weights()

    def _init_weights(self):
        nn.init.normal_(self.user_emb.weight, std=0.1)
        nn.init.normal_(self.item_emb.weight, std=0.1)

    def forward(self, user, item_i, item_j):
        # Compute difference between positive and negative item scores
        u = self.user_emb(user)
        i = self.item_emb(item_i)
        j = self.item_emb(item_j)
        x_ui = (u * i).sum(dim=1)
        x_uj = (u * j).sum(dim=1)
        return x_ui - x_uj

    def predict(self, user, item):
        # Compute preference score for a single item
        u = self.user_emb(user)
        i = self.item_emb(item)
        return (u * i).sum(dim=1)

In [ ]:
class BPRTrainer:
    def __init__(self, num_users, num_items, num_factors=32, lr=0.01, reg=1e-4, epochs=10,
                 device=device, outputs_path="outputs", batch_size=1024):
        self.device = device
        self.outputs_path = os.path.join(outputs_path, 'BPR')
        os.makedirs(self.outputs_path, exist_ok=True)
        self.model_file = os.path.join(self.outputs_path, "bpr_model.pt")
        self.batch_size = batch_size

        self.model = BPRModel(num_users, num_items, num_factors).to(device)
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, weight_decay=reg)
        self.epochs = epochs

    def _build_user_pos(self, train_data):
        user_pos = defaultdict(set)
        for _, row in train_data.iterrows():
            user_pos[int(row['user_id']) - 1].add(int(row['item_id']) - 1)
        return user_pos

    def train(self, train_data):
        user_pos = self._build_user_pos(train_data)
        num_items = self.model.item_emb.num_embeddings
        all_items = np.arange(num_items)

        print("Training BPR with mini-batch...")
        for epoch in range(self.epochs):
            batch_u, batch_i, batch_j = [], [], []
            total_loss = 0
            for u in user_pos:
                pos_items = list(user_pos[u])
                if len(pos_items) == 0:
                    continue
                for i in pos_items:
                    # sample negative
                    j = np.random.choice(np.setdiff1d(all_items, pos_items))
                    batch_u.append(u)
                    batch_i.append(i)
                    batch_j.append(j)
                    # train in batch
                    if len(batch_u) >= self.batch_size:
                        loss = self._train_batch(batch_u, batch_i, batch_j)
                        total_loss += loss
                        batch_u, batch_i, batch_j = [], [], []
            # train remaining
            if batch_u:
                loss = self._train_batch(batch_u, batch_i, batch_j)
                total_loss += loss

            print(f"Epoch {epoch+1}/{self.epochs}, Avg Loss={total_loss/len(user_pos):.4f}")

        torch.save(self.model.state_dict(), self.model_file)
        self.trained = True
        print(f"BPR model saved to {self.model_file}")

    def _train_batch(self, batch_u, batch_i, batch_j):
        batch_u = torch.LongTensor(batch_u).to(self.device)
        batch_i = torch.LongTensor(batch_i).to(self.device)
        batch_j = torch.LongTensor(batch_j).to(self.device)

        x_uij = self.model(batch_u, batch_i, batch_j)
        loss = -torch.log(torch.sigmoid(x_uij)).mean()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def recommend(self, user_id, train_data=None, top_k=10):
        self.model.eval()
        u = torch.LongTensor([user_id - 1]).to(self.device)
        scores = self.model.predict(u, torch.arange(self.model.item_emb.num_embeddings).to(self.device)).cpu().detach().numpy()
        if train_data is not None:
            interacted = set(train_data[train_data['user_id'] == user_id]['item_id'])
        else:
            interacted = set()
        candidates = [(i+1, s) for i, s in enumerate(scores) if (i+1) not in interacted]
        candidates.sort(key=lambda x: x[1], reverse=True)
        return candidates[:top_k]

### BPR Hyperparam Searching and Training

> Since BPR is a ranking task, nDCG/MRR/Precision/Recall can be used as target. However, `loss` is used here for simplicity.

In [ ]:
def hyperparameter_search_BPR(train_data, num_users, num_items, device, OUTPUTS_PATH, load_best_only=False):
    """
    Perform grid search over BPR hyperparameters.
    If load_best_only=True, directly load the previously best model without retraining.
    """

    import json

    bpr_dir = os.path.join(OUTPUTS_PATH, "BPR")
    os.makedirs(bpr_dir, exist_ok=True)

    best_config_file = os.path.join(bpr_dir, "best_bpr_config.json")
    best_model_file = os.path.join(bpr_dir, "bpr_model.pt")

    # ===============================
    # Case 1: Only load best model
    # ===============================
    if load_best_only:
        if not os.path.exists(best_config_file) or not os.path.exists(best_model_file):
            raise FileNotFoundError("⚠️ No saved best BPR model found. Run grid search first.")

        with open(best_config_file, "r") as f:
            best_result = json.load(f)

        print(f"🔹 Loading best BPR model: {best_result}")

        best_num_factors = best_result["params"][0]
        best_lr = best_result["params"][1]
        best_reg = best_result["params"][2]

        bpr_trainer = BPRTrainer(
            num_users, num_items,
            num_factors=best_num_factors, lr=best_lr, reg=best_reg,
            epochs=10, device=device, outputs_path=OUTPUTS_PATH
        )

        bpr_trainer.model.load_state_dict(torch.load(best_model_file, map_location=str(device)))
        print(f"✅ Loaded pre-trained BPR model from {best_model_file}")

        print("BPR Top-5 Recommendations:", bpr_trainer.recommend(1, train_data=train_data, top_k=5))
        return bpr_trainer

    # ===============================
    # Case 2: Full grid search
    # ===============================
    bpr_param_grid = {
        "num_factors": [16, 32, 64],
        "lr": [0.005, 0.01],
        "reg": [1e-4, 1e-3],
    }

    best_result = {"loss": float("inf"), "params": None}

    print("\n=== BPR Hyperparameter Search ===")
    for num_factors, lr, reg in itertools.product(
            bpr_param_grid["num_factors"],
            bpr_param_grid["lr"],
            bpr_param_grid["reg"]):

        print(f"\nTraining BPR with num_factors={num_factors}, lr={lr}, reg={reg}")
        bpr_trainer = BPRTrainer(
            num_users, num_items,
            num_factors=num_factors, lr=lr, reg=reg,
            epochs=5, device=device, outputs_path=OUTPUTS_PATH
        )
        bpr_trainer.train(train_data)

        # ---- Compute average training loss as proxy metric ----
        user_pos = bpr_trainer._build_user_pos(train_data)
        total_loss = 0
        num_samples = 0

        for u in list(user_pos.keys())[:200]:  # Sample subset for speed
            pos_items = list(user_pos[u])
            if len(pos_items) == 0:
                continue
            j = np.random.randint(0, num_items)
            u_tensor = torch.LongTensor([u]).to(device)
            i_tensor = torch.LongTensor([pos_items[0]]).to(device)
            j_tensor = torch.LongTensor([j]).to(device)

            with torch.no_grad():
                x_uij = bpr_trainer.model(u_tensor, i_tensor, j_tensor)
                loss = -torch.log(torch.sigmoid(x_uij)).mean().item()

            total_loss += loss
            num_samples += 1

        avg_loss = total_loss / num_samples
        print(f"Validation Avg Loss = {avg_loss:.4f}")

        if avg_loss < best_result["loss"]:
            best_result["loss"] = avg_loss
            best_result["params"] = (num_factors, lr, reg)

    print(f"\n✅ Best BPR Config: factors={best_result['params'][0]}, "
          f"lr={best_result['params'][1]}, reg={best_result['params'][2]}, "
          f"Loss={best_result['loss']:.4f}")

    # Save best configuration to JSON file
    with open(best_config_file, "w") as f:
        json.dump(best_result, f, indent=4)
        print(f"💾 Saved best BPR configuration to {best_config_file}")

    # ===============================
    # Train final best model
    # ===============================
    best_num_factors, best_lr, best_reg = best_result["params"]
    final_bpr = BPRTrainer(
        num_users, num_items,
        num_factors=best_num_factors, lr=best_lr, reg=best_reg,
        epochs=10, device=device, outputs_path=OUTPUTS_PATH
    )
    final_bpr.train(train_data)

    # Save best model as fixed filename
    torch.save(final_bpr.model.state_dict(), best_model_file)
    print(f"💾 Best BPR model saved to {best_model_file}")

    print("BPR Top-5 Recommendations:", final_bpr.recommend(1, train_data=train_data, top_k=5))
    return final_bpr

In [ ]:
# bpr_model = hyperparameter_search_BPR(train_data, num_users, num_items, device, OUTPUTS_PATH)
bpr_model = hyperparameter_search_BPR(train_data, num_users, num_items, device, OUTPUTS_PATH, load_best_only=True)  # debug only

In [ ]:
target_user = 1
k = 10

recommendations = bpr_model.recommend(
    target_user, train_data, k
)

print(f"Top-{k} recommendations based on Bayesian Personalized Ranking for user {target_user}:")
print(recommendations)

## 1.6 Hybrid recommender

### 1.6.1 Hybrid model for rating

#### Generate component predictions

The hybrid recommender combines the rating predictions from multiple base models ($C_j$) to generate a final rating.
Each base model gives a predicted rating $score_{C_j}(u,i)$ for a user $u$ and item $i$:
$$
score_{C_j}(u,i)
$$
The hybrid model learns how to **weight these base model predictions** to produce a final score:
$$
score(u,i) = \sum_{j=1}^k \alpha_j \cdot score_{C_j}(u,i)
$$
where the $(\alpha_j)$ coefficients represent the weight learned for each component, showing how much each model contributes to the final hybrid prediction.

In [ ]:
def generate_component_predictions(val_data, train_data,
                                   item_similarity_matrix, user_similarity_matrix,
                                   mf_model,
                                   k_item=best_k_item, k_user=best_k_user,
                                   device=device, verbose=False):
    """
    Generate predictions from multiple recommender components for each (user, item) pair.

    Handles both 0-indexed and 1-indexed matrices automatically.
    Returns DataFrame with columns:
    [user_id, item_id, true_rating, cb, userknn, itemknn, mf]
    """

    preds = []
    skipped = 0

    # Detect index offset automatically (0-indexed or 1-indexed)
    user_min = user_similarity_matrix.index.min()
    item_min = item_similarity_matrix.index.min()

    user_offset = 1 if user_min == 0 else 0
    item_offset = 1 if item_min == 0 else 0

    global_mean_rating = train_data["rating"].mean()

    if pd.isna(global_mean_rating):
        print("!!! CRITICAL ERROR: global_mean_rating is NaN !!!")
        print("This means your train_data['rating'] column might be all NaNs.")
        print("Dropping NaNs from training data for mean calculation...")
        # Try a fallback mean calculation
        global_mean_rating = train_data["rating"].dropna().mean()
        if pd.isna(global_mean_rating):
            raise ValueError(
                "CRITICAL: Still cannot calculate global_mean_rating. Check train_data."
            )

    if verbose:
        print(f"Detected user matrix offset: {user_offset}, item matrix offset: {item_offset}")

    for idx, row in val_data.iterrows():
        user_id, item_id, true_rating = int(row['user_id']), int(row['item_id']), row['rating']

        try:
            # --- Content-based prediction (using full embedding, average aggregation) ---
            cb_pred = get_user_item_prediction_content_based(train_data, user_id, item_id, 'full', 'weighted_avg')

            # --- User-based CF prediction ---
            adj_user_id = user_id - user_offset
            if adj_user_id not in user_similarity_matrix.index:
                raise KeyError(f"User {adj_user_id} not in similarity matrix")
            ub_pred = predict_rating_user_based(
                train_data,
                user_similarity_matrix,
                user_id,
                item_id,
                global_mean_rating, k=k_user,
            )

            # --- Item-based CF prediction ---
            adj_item_id = item_id - item_offset
            if adj_item_id not in item_similarity_matrix.index:
                raise KeyError(f"Item {adj_item_id} not in similarity matrix")
            ib_pred = predict_rating_item_based(train_data, item_similarity_matrix, user_id, item_id, k=k_item)

            # --- Matrix Factorization prediction ---
            mf_pred = mf_model.predict(user_id, item_id)

            # --- BPR prediction ---
            # with torch.no_grad():
            #     bpr_model.model.to(device)
            #     bpr_pred = bpr_model.model.predict(
            #         torch.LongTensor([user_id - 1]).to(device),
            #         torch.LongTensor([item_id - 1]).to(device)
            #     ).item()

            preds.append([user_id, item_id, true_rating, cb_pred, ub_pred, ib_pred, mf_pred])

        except Exception as e:
            skipped += 1
            if verbose:
                print(f"⚠️ Skipped user {user_id}, item {item_id}: {e}")
            continue

    df_preds = pd.DataFrame(preds, columns=['user_id', 'item_id', 'true_rating', 'cb', 'userknn', 'itemknn', 'mf'])
    print(f"✅ Completed hybrid prediction data generation. Skipped {skipped} problematic pairs.")
    print(f"Returned shape: {df_preds.shape}")
    return df_preds

#### Train the hybrid regression model

In [ ]:
from sklearn.linear_model import LinearRegression

# Generate predictions for validation set
hybrid_train = generate_component_predictions(
    val_data, train_data,
    item_similarity_matrix, user_similarity_matrix,
    mf_model,
    k_item=best_k_item, k_user=best_k_user,
    verbose=True
)

X = hybrid_train[['cb','userknn','itemknn','mf']]
y = hybrid_train['true_rating']

# Train regression model
hybrid_reg = LinearRegression(fit_intercept=True)

hybrid_reg.fit(X, y)

# Display learned coefficients
coefficients = pd.DataFrame({
    'Component': X.columns,
    'Alpha (Weight)': hybrid_reg.coef_
})
display(coefficients)
print(f"Intercept (bias): {hybrid_reg.intercept_:.4f}")


In [ ]:
print(hybrid_train.isna().sum())

#### Check ranges and summary stats

In [ ]:
# Look at the min, max, mean, std for each component
hybrid_train[['cb','userknn','itemknn','mf']].describe().T

#### Train the regression after scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select only numeric prediction columns
X = hybrid_train[['cb','userknn','itemknn','mf']]

# Fit the scaler on training data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train regression on scaled features
hybrid_reg_scaled = LinearRegression()
hybrid_reg_scaled.fit(X_scaled, hybrid_train['true_rating'])

# Show new coefficients
coefficients_scaled = pd.DataFrame({
    'Component': X.columns,
    'Alpha (Weight)': hybrid_reg_scaled.coef_
})
display(coefficients_scaled)
print(f"Intercept (bias): {hybrid_reg_scaled.intercept_:.4f}")

Before scaling the predictors have different magnitudes. The regression absorbs those differences into both coefficients and intercept so the small negative intercept just compensates for overly high CB predictions. After scaling, all predictors are centered (mean=0) and standardized (std=1) so the intercept becomes the mean of the target, i.e. the average movie rating.

#### Make hybrid predictions on test set (maybe move this to Task 2)


In [ ]:
def evaluate_hybrid_models(
    test_data,
    train_data,
    item_similarity_matrix,
    user_similarity_matrix,
    mf_model,
    hybrid_reg_scaled,
    bpr_model,
    k_item=best_k_item,
    k_user=best_k_user,
    device=device,
):
    # Generate component predictions on test data
    hybrid_test = generate_component_predictions(
        test_data,
        train_data,
        item_similarity_matrix,
        user_similarity_matrix,
        mf_model,
        # bpr_model,  # Fixed: no BPR in rating prediction task
        k_item=k_item,
        k_user=k_user,
        device=device,
        verbose=False,
    )

    # Predict using learned hybrid regression
    X_test = hybrid_test[["cb", "userknn", "itemknn", "mf"]]
    y_test = hybrid_test["true_rating"]
    y_pred = hybrid_reg.predict(X_test)

    # Evaluate
    test_rmse = RMSE(y_test, y_pred)
    print(f"✅ Hybrid Test RMSE = {test_rmse:.4f}")

    X_test_scaled = scaler.transform(hybrid_test[["cb", "userknn", "itemknn", "mf"]])
    y_pred_scaled = hybrid_reg_scaled.predict(X_test_scaled)
    test_rmse_scaled = RMSE(hybrid_test["true_rating"], y_pred_scaled)
    print(f"✅ Scaled Hybrid Test RMSE = {test_rmse_scaled:.4f}")

    return test_rmse_scaled

rmse_hybrid = evaluate_hybrid_models(
    test_data,
    train_data,
    item_similarity_matrix,
    user_similarity_matrix,
    mf_model,
    hybrid_reg_scaled,
    bpr_model,
    k_item=best_k_item,
    k_user=best_k_user,
    device=device,
)

> Although scaling did not change RMSE (since linear regression predictions are invariant to affine scaling of inputs), it standardized the component feature space. This gives interpretable weights (comparable across models) and ensures numerical stability. BPR is almost non significant because it is for ranking, not rating prediction.

#### Visualize coefficient importance

In [ ]:
plt.figure(figsize=(7,4))
sns.barplot(data=coefficients_scaled, x='Component', y='Alpha (Weight)', palette='coolwarm')
plt.title('Hybrid Model Component Importance (After Scaling)')
plt.ylabel('Weight (α)')
plt.show()


In [ ]:
coeff_compare = pd.DataFrame({
    'Component': X.columns,
    'Unscaled': hybrid_reg.coef_,
    'Scaled': hybrid_reg_scaled.coef_
}).melt(id_vars='Component', var_name='Model', value_name='Weight (α)')

plt.figure(figsize=(7,4))
sns.barplot(data=coeff_compare, x='Component', y='Weight (α)', hue='Model', palette='coolwarm')
plt.title('Hybrid Coefficients Before vs After Scaling')
plt.show()


### 1.6.2 Hybrid model for ranking

#### Construct pairwise preference dataset

For ranking tasks, we optimize using ***pairwise*** preferences (similar to **BPR**). For each user $u$, we create triplets $(u, i, j)$ where:
- $i$ is a positively rated item (rating ≥ 4)
- $j$ is a negatively sampled item (not rated or rating < 4)

The hybrid ranking model learns to maximize:
$$\mathcal{L} = -\log \sigma(f(x_{ui}) - f(x_{uj}))$$

where $f(x_{ui})$ is the weighted combination of component predictions.

In [ ]:
def generate_pairwise_preferences(train_data, user_sample_ratio=0.3, neg_samples_per_pos=1):
    """
    Generate pairwise preference dataset for hybrid ranking model training.
    
    Args:
        train_data (pd.DataFrame): Training ratings data
        user_sample_ratio (float): Proportion of users to sample
        neg_samples_per_pos (int): Number of negative items per positive item
    
    Returns:
        pd.DataFrame: Pairwise preferences with columns [user_id, item_i, item_j]
                     where item_i is positive, item_j is negative
    """
    pairwise_data = []
    
    # Sample users for efficiency
    all_users = train_data['user_id'].unique()
    sampled_users = np.random.choice(all_users, size=int(len(all_users) * user_sample_ratio), replace=False)
    
    all_items = set(train_data['item_id'].unique())
    
    for user_id in tqdm(sampled_users, desc="Generating pairwise preferences"):
        user_ratings = train_data[train_data['user_id'] == user_id]
        
        # Positive items: rating >= 4
        pos_items = set(user_ratings[user_ratings['rating'] >= 4]['item_id'])
        
        # Negative candidates: all items - positive items
        neg_candidates = list(all_items - pos_items)
        
        if not pos_items or not neg_candidates:
            continue
        
        # Create pairs
        for pos_item in pos_items:
            # Sample negative items
            num_neg = min(neg_samples_per_pos, len(neg_candidates))
            neg_items = np.random.choice(neg_candidates, size=num_neg, replace=False)
            
            for neg_item in neg_items:
                pairwise_data.append([user_id, pos_item, neg_item])
    
    df_pairs = pd.DataFrame(pairwise_data, columns=['user_id', 'item_i', 'item_j'])
    print(f"✅ Generated {len(df_pairs)} pairwise preferences from {len(sampled_users)} users")
    return df_pairs

In [ ]:
# Generate pairwise preference dataset
pairwise_train = generate_pairwise_preferences(train_data, user_sample_ratio=0.3, neg_samples_per_pos=2)
print(f"Pairwise dataset shape: {pairwise_train.shape}")
display(pairwise_train.head())

#### Generate component predictions for pairwise data

In [ ]:
def generate_component_predictions_pairwise(pairwise_data, train_data,
                                            item_similarity_matrix, user_similarity_matrix,
                                            mf_model, bpr_model,
                                            k_item=20, k_user=70,
                                            device='cpu', verbose=False):
    """
    Generate component predictions for pairwise preference data.
    
    Returns DataFrame with columns:
    [user_id, item_i, item_j, cb_i, cb_j, userknn_i, userknn_j, ..., bpr_i, bpr_j]
    """
    preds = []
    skipped = 0
    
    global_mean_rating = train_data["rating"].mean()
    
    for idx, row in tqdm(pairwise_data.iterrows(), total=len(pairwise_data), desc="Generating component predictions"):
        user_id = int(row['user_id'])
        item_i = int(row['item_i'])
        item_j = int(row['item_j'])
        
        try:
            # --- Predictions for positive item i ---
            cb_i = get_user_item_prediction_content_based(train_data, user_id, item_i, 'full', 'weighted_avg')
            ub_i = predict_rating_user_based(train_data, user_similarity_matrix, user_id, item_i, global_mean_rating, k=k_user)
            ib_i = predict_rating_item_based(train_data, item_similarity_matrix, user_id, item_i, k=k_item)
            mf_i = mf_model.predict(user_id, item_i)
            with torch.no_grad():
                bpr_i = bpr_model.model.predict(
                    torch.LongTensor([user_id - 1]).to(device),
                    torch.LongTensor([item_i - 1]).to(device)
                ).item()
            
            # --- Predictions for negative item j ---
            cb_j = get_user_item_prediction_content_based(train_data, user_id, item_j, 'full', 'weighted_avg')
            ub_j = predict_rating_user_based(train_data, user_similarity_matrix, user_id, item_j, global_mean_rating, k=k_user)
            ib_j = predict_rating_item_based(train_data, item_similarity_matrix, user_id, item_j, k=k_item)
            mf_j = mf_model.predict(user_id, item_j)
            with torch.no_grad():
                bpr_j = bpr_model.model.predict(
                    torch.LongTensor([user_id - 1]).to(device),
                    torch.LongTensor([item_j - 1]).to(device)
                ).item()
            
            preds.append([user_id, item_i, item_j, 
                         cb_i, cb_j, ub_i, ub_j, ib_i, ib_j, mf_i, mf_j, bpr_i, bpr_j])
            
        except Exception as e:
            skipped += 1
            if verbose:
                print(f"⚠️ Skipped user {user_id}, items ({item_i}, {item_j}): {e}")
            continue
    
    df_preds = pd.DataFrame(preds, columns=[
        'user_id', 'item_i', 'item_j',
        'cb_i', 'cb_j', 'userknn_i', 'userknn_j', 'itemknn_i', 'itemknn_j',
        'mf_i', 'mf_j', 'bpr_i', 'bpr_j'
    ])
    print(f"✅ Completed pairwise prediction generation. Skipped {skipped} problematic pairs.")
    print(f"Returned shape: {df_preds.shape}")
    return df_preds

In [ ]:
# Generate component predictions for pairwise data
hybrid_ranking_train = generate_component_predictions_pairwise(
    pairwise_train, train_data,
    item_similarity_matrix, user_similarity_matrix,
    mf_model, bpr_model,
    k_item=20, k_user=70,
    device=device,
    verbose=False
)

print(f"Pairwise predictions shape: {hybrid_ranking_train.shape}")
display(hybrid_ranking_train.head())

#### Train hybrid ranking model with BPR loss

In [ ]:
class HybridRankingModel:
    """
    Hybrid ranking model that learns to combine component predictions using BPR loss.
    """
    def __init__(self, num_components=5, lr=0.01, reg=1e-4, epochs=20):
        self.num_components = num_components
        self.lr = lr
        self.reg = reg
        self.epochs = epochs
        
        # Initialize weights (alphas) randomly
        self.weights = np.random.randn(num_components) * 0.1
        self.scaler = StandardScaler()
    
    def _compute_score(self, features):
        """Compute hybrid score as weighted sum of components"""
        return np.dot(features, self.weights)
    
    def _sigmoid(self, x):
        """Numerically stable sigmoid"""
        return np.where(
            x >= 0,
            1 / (1 + np.exp(-x)),
            np.exp(x) / (1 + np.exp(x))
        )
    
    def fit(self, X_i, X_j):
        """
        Train using pairwise BPR loss.
        
        Args:
            X_i: features for positive items (N, num_components)
            X_j: features for negative items (N, num_components)
        """
        # Fit scaler on combined data
        all_features = np.vstack([X_i, X_j])
        self.scaler.fit(all_features)
        
        # Scale features
        X_i_scaled = self.scaler.transform(X_i)
        X_j_scaled = self.scaler.transform(X_j)
        
        n_samples = len(X_i_scaled)
        
        print(f"Training hybrid ranking model for {self.epochs} epochs...")
        for epoch in range(self.epochs):
            total_loss = 0
            
            # Compute scores
            scores_i = self._compute_score(X_i_scaled)
            scores_j = self._compute_score(X_j_scaled)
            
            # BPR loss: -log(sigmoid(score_i - score_j))
            x_uij = scores_i - scores_j
            sigmoid_uij = self._sigmoid(x_uij)
            
            # Compute loss (add epsilon to avoid log(0))
            loss = -np.mean(np.log(sigmoid_uij + 1e-10))
            total_loss = loss
            
            # Compute gradient
            # d/dw[-log(sigmoid(x_uij))] = -(1 - sigmoid(x_uij)) * (X_i - X_j)
            grad_coeff = -(1 - sigmoid_uij).reshape(-1, 1)
            grad = np.mean(grad_coeff * (X_i_scaled - X_j_scaled), axis=0)
            
            # Add L2 regularization gradient
            grad += self.reg * self.weights
            
            # Update weights
            self.weights -= self.lr * grad
            
            if (epoch + 1) % 5 == 0:
                print(f"Epoch {epoch+1}/{self.epochs}, Loss={total_loss:.4f}")
        
        print(f"✅ Training complete. Final loss: {total_loss:.4f}")
    
    def predict(self, X):
        """Predict scores for given features"""
        X_scaled = self.scaler.transform(X)
        return self._compute_score(X_scaled)
    
    def get_weights(self):
        """Return learned weights"""
        return self.weights

In [ ]:
# Prepare training data
component_names = ['cb', 'userknn', 'itemknn', 'mf', 'bpr']

# Features for positive items
X_i = hybrid_ranking_train[['cb_i', 'userknn_i', 'itemknn_i', 'mf_i', 'bpr_i']].values

# Features for negative items
X_j = hybrid_ranking_train[['cb_j', 'userknn_j', 'itemknn_j', 'mf_j', 'bpr_j']].values

# Train hybrid ranking model
hybrid_rank_model = HybridRankingModel(num_components=5, lr=0.01, reg=1e-4, epochs=100)
hybrid_rank_model.fit(X_i, X_j)

# Display learned weights
learned_weights = hybrid_rank_model.get_weights()
hybrid_ranking_weights_df = pd.DataFrame({
    'Component': component_names,
    'Weight (α)': learned_weights
})
print("\n--- Learned Component Weights for Ranking ---")
display(hybrid_ranking_weights_df)

#### Visualize learned ranking weights

In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(data=hybrid_ranking_weights_df, x='Component', y='Weight (α)', palette='viridis')
plt.title('Hybrid Ranking Model Component Weights (BPR Loss)')
plt.ylabel('Weight (α)')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.3)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

#### Generate Top-K recommendations using trained hybrid ranking model

In [ ]:
def recommend_topk_hybrid_ranking(user_id, hybrid_rank_model, train_data, item_similarity_matrix, user_similarity_matrix,
                                  mf_model, bpr_model, k_item=20, k_user=70, top_k=10, device='cpu'):
    """
    Generate Top-K recommendations using the trained hybrid ranking model.
    
    Args:
        user_id (int): Target user.
        hybrid_rank_model: Trained HybridRankingModel.
        train_data (pd.DataFrame): Training ratings.
        item_similarity_matrix (pd.DataFrame): Precomputed item-item similarities.
        user_similarity_matrix (pd.DataFrame): Precomputed user-user similarities.
        mf_model: Trained MF model.
        bpr_model: Trained BPR model.
        k_item (int): Number of neighbors for ItemKNN.
        k_user (int): Number of neighbors for UserKNN.
        top_k (int): Number of items to recommend.
        device (str): 'cpu' or 'cuda'.
    
    Returns:
        list of (item_id, hybrid_score)
    """
    global_mean_rating = train_data["rating"].mean()
    
    # Get items the user already rated
    user_rated_items = set(train_data[train_data['user_id'] == user_id]['item_id'])
    all_items = set(train_data['item_id'].unique())
    candidate_items = all_items - user_rated_items
    
    predictions = []
    for item_id in candidate_items:
        try:
            # --- Get component predictions ---
            cb_pred = get_user_item_prediction_content_based(train_data, user_id, item_id, 'full', 'weighted_avg')
            ub_pred = predict_rating_user_based(train_data, user_similarity_matrix, user_id, item_id, global_mean_rating, k=k_user)
            ib_pred = predict_rating_item_based(train_data, item_similarity_matrix, user_id, item_id, k=k_item)
            mf_pred = mf_model.predict(user_id, item_id)
            with torch.no_grad():
                bpr_model.model.to(device)
                bpr_pred = bpr_model.model.predict(
                    torch.LongTensor([user_id - 1]).to(device),
                    torch.LongTensor([item_id - 1]).to(device)
                ).item()
            
            # Prepare feature vector for hybrid ranking model
            features = np.array([[cb_pred, ub_pred, ib_pred, mf_pred, bpr_pred]])
            hybrid_score = hybrid_rank_model.predict(features)[0]
            
            predictions.append((item_id, hybrid_score))
            
        except Exception as e:
            # Skip if prediction not possible
            continue
    
    # Sort by predicted score descending
    recommendations = sorted(predictions, key=lambda x: x[1], reverse=True)[:top_k]
    return recommendations

# Task 2) Experiments for both rating prediction and ranking tasks, and conducting offline evaluation

In [ ]:
def Precision(ground_truth, rec_list):
    # Implement a function that computes Precision across ground truth data and recommendation list generated for each user.
    # Note that ground_truth and rec_list contain the list of items for all users, e.g., 2-dimensional arrays.

    result = 0.0


    ############# Your code here ############
    # Precision measures how many of the positive samples are actually positive
    # so how many of the recommended items are actually relevant

    precisions = []

    for gt, rec in zip(ground_truth, rec_list):
        if not rec:
            continue
        hits = len(set(gt) & set(rec))
        precision = hits / len(rec)
        precisions.append(precision)

    result = np.mean(precisions)
    #########################################

    return result

def Recall(ground_truth, rec_list):
    # Implement a function that computes Recall across ground truth data and recommendation list generated for each user.
    # Note that ground_truth and rec_list contain the list of items for all users, e.g., 2-dimensional arrays.

    result = 0.0

    ############# Your code here ############
    # recall is out of all relevant items, how many did it positively predict
    recalls = []

    for gt, rec in zip(ground_truth, rec_list):
        if not rec or not gt:
            continue
        hits = len(set(gt) & set(rec))
        recall = hits / len(gt)
        recalls.append(recall)

    result = np.mean(recalls)

    #########################################

    return result

def NDCG(ground_truth, rec_list):
    # Implement a function that computes NDCG across ground truth data and recommendation list generated for each user.
    # Note that ground_truth and rec_list contain the list of items for all users, e.g., 2-dimensional arrays.

    result = 0.0

    ############# Your code here ############
    ndcgs = []
    for gt_items, rec_items in zip(ground_truth, rec_list):
        dcg = 0.0

        # Calculate DCG
        for i, item in enumerate(rec_items):
            if item in gt_items:
                dcg += 1.0 / np.log2(i + 1)

        # Calculate IDCG
        idcg = 0.0
        for i in range(min(len(rec_items), len(gt_items))):
            idcg += 1.0 / np.log2(i + 1)

        # Calculate NDCG
        if idcg == 0:
            ndcgs.append(0.0)
        else:
            ndcgs.append(dcg / idcg)

    return np.mean(ndcgs)
    #########################################

In [ ]:
# -----------------------------
# Ranking Metrics "@k"
# -----------------------------
def precision_at_k(actual, predicted, k=10):
    """Precision@K"""
    if len(predicted) > k:
        predicted = predicted[:k]
    actual_set = set(actual)
    predicted_set = set(predicted)
    return len(actual_set & predicted_set) / float(k) if k else 0.0


def recall_at_k(actual, predicted, k=10):
    """Recall@K"""
    actual_set = set(actual)
    predicted_set = set(predicted[:k])
    return len(actual_set & predicted_set) / float(len(actual_set)) if actual_set else 0.0


def ndcg_at_k(actual, predicted, k=10):
    """Normalized Discounted Cumulative Gain@K"""
    if not actual:
        return 0.0
    predicted = predicted[:k]
    dcg = 0.0
    for idx, item in enumerate(predicted):
        if item in actual:
            dcg += 1.0 / np.log2(idx + 2)
    ideal_dcg = sum(1.0 / np.log2(i + 2) for i in range(min(len(actual), k)))
    return dcg / ideal_dcg if ideal_dcg > 0 else 0.0


def average_precision_at_k(actual, predicted, k=10):
    """Mean Average Precision@K"""
    if not actual:
        return 0.0
    predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    return score / min(len(actual), k)

## 2.1 Rating Prediction Task Evaluation

In [ ]:
# We computed this when we optimized hyperparameters earlier but here we go again
# RMSE of content-based, user-based, item-based, and matrix factorization model as well as the hybrid model


if rmse_content_based and rmse_user_based and rmse_item_based and rmse_mf and rmse_hybrid:
    print("Skipping evaluation, already computed.")
    print(f"Content-based RMSE: {rmse_content_based}")
    print(f"User-based RMSE: {rmse_user_based}")
    print(f"Item-based RMSE: {rmse_item_based}")
    print(f"Matrix Factorization RMSE: {rmse_mf}")  
    print(f"Hybrid RMSE: {rmse_hybrid}")

else:
    print("Running Content rating based evaluations on test data...")
    rmse_content_based = evaluate_rating_prediction_content_based(
                train_data,
                test_data,
                "full",
                "weighted_avg")

    print("Running user based rating evaluations on test data...")
    rmse_user_based = evaluate_rating_prediction_user_based(train_data, test_data, user_similarity_matrix, 70)

    print("Running item based rating evaluations on test data...")
    rmse_item_based = evaluate_rating_prediction_item_based(train_data, test_data, item_similarity_matrix, 20)

    print("Running matrix factorization rating evaluations on test data...")
    rmse_mf = mf_model.evaluate(test_data)

    print("Running hybrid rating evaluations on test data...")
    rmse_hybrid = evaluate_hybrid_models(
        test_data,
        train_data,
        item_similarity_matrix,
        user_similarity_matrix,
        mf_model,
        hybrid_reg_scaled,
        bpr_model,
        k_item=20,
        k_user=50,
        device=device,
    )

    print(type(rmse_content_based), rmse_content_based)
    print(type(rmse_user_based), rmse_user_based)
    print(type(rmse_item_based), rmse_item_based)
    print(type(rmse_mf), rmse_mf)
    print(type(rmse_hybrid), rmse_hybrid)

    # --- Final Comparison ---
    print("\n--- Model RMSE Comparison ---")
    results = {
        "Content-Based": rmse_content_based,
        "User-Based KNN": rmse_user_based,
        "Item-Based KNN": rmse_item_based,
        "Matrix Factorization": rmse_mf,
        "Hybrid (Scaled)": rmse_hybrid,
    }
    results_df = pd.DataFrame(list(results.items()), columns=["Model", "RMSE"])
    results_df = results_df.sort_values("RMSE").set_index("Model")
    display(results_df)

## 2.2 Ranking Prediction Task Evaluation

In [ ]:
def evaluate_all_models_ranking(
    train_data: pd.DataFrame,
    test_data: pd.DataFrame,
    user_similarity_matrix: pd.DataFrame,
    item_similarity_matrix: pd.DataFrame,
    mf_model,  
    bpr_model,
    k: int = 10,
) -> pd.DataFrame:
    """
    Evaluates all five models against the test data.
    Model objects are passed as direct parameters.
    Hyperparameters (k_user, k_item, etc.) are hardcoded inside.
    """

    test_users = test_data["user_id"].unique()

    # Data structures to hold results
    all_ground_truths = []

    # Define the models we are testing
    model_names = [
        "Content-Based",
        "User-Based KNN",
        "Item-Based KNN",
        "Matrix Factorization",
        "BPR Model",
    ]

    all_recommendations = {model_name: [] for model_name in model_names}

    for user_id in tqdm(test_users, desc="Evaluating all models via ranking task"):

        # Get all the items that the user has rated 4.0 or higher in the test set
        gt = list(test_data[(test_data['user_id'] == user_id) & (test_data['rating'] >= 4.0)]['item_id'])

        if not gt:
            continue

        all_ground_truths.append(gt)

        # Get recommendations from each one of the model

        # --- Content-Based ---
        recs_cb = recommend_topk_content_based(
            train_data=train_data,
            target_user=user_id,
            k=k,
            content_type="full",  
            aggregation_method="weighted_avg", 
        )

        recs_cb_ids = [item_id for item_id, score in recs_cb]

        # print(f"User {user_id} Content-Based Recs: {recs_cb[:5]}")
        all_recommendations["Content-Based"].append(recs_cb_ids)

        # --- User-Based KNN ---
        recs_ub = recommend_topk_user_based(
            train_data=train_data,
            target_user=user_id,
            k=k,
            user_similarity_matrix=user_similarity_matrix,
            k_user=50,  
        )

        recs_ub_ids = [item_id for item_id, score in recs_ub]

        # print(f"User {user_id} User-Based KNN Recs: {recs_ub[:5]}")
        all_recommendations["User-Based KNN"].append(recs_ub_ids)

        # --- Item-Based KNN ---
        recs_ib = recommend_topk_item_based(
            train_data=train_data,
            target_user=user_id,
            k=k,
            item_similarity_matrix=item_similarity_matrix,
            k_item=20,  
        )

        recs_ib_ids = [item_id for item_id, score in recs_ib]
        # print(f"User {user_id} Item-Based KNN Recs: {recs_ib[:5]}")
        all_recommendations["Item-Based KNN"].append(recs_ib_ids)

        # --- Matrix Factorization ---
        recs_mf = recommend_topk_mf(
            mf_trainer=mf_model, train_data=train_data, target_user=user_id, k=k
        )
        # print(f"User {user_id} Matrix Factorization Recs: {recs_mf[:5]}")
        all_recommendations["Matrix Factorization"].append(recs_mf)

        # --- BPR Model ---
        recs_bpr = bpr_model.recommend(target_user, train_data, k)
        # print(f"User {user_id} BPR Model Recs: {recs_bpr[:5]}")

        recs_bpr_ids = [item_id for item_id, score in recs_bpr]
        all_recommendations["BPR Model"].append(recs_bpr_ids)

    print("\nCalculating final metrics...")

    final_results = {}
    for model_name, rec_lists in all_recommendations.items():
        if not rec_lists:
            print(f"Warning: No recommendations generated for {model_name}")
            continue

        prec = Precision(all_ground_truths, rec_lists)
        rec = Recall(all_ground_truths, rec_lists)
        ndcg = NDCG(all_ground_truths, rec_lists)

        final_results[model_name] = {
            "Precision@K": prec,
            "Recall@K": rec,
            "NDCG@K": ndcg,
        }

    results_df = pd.DataFrame.from_dict(final_results, orient="index")
    results_df = results_df.sort_values("NDCG@K", ascending=False)

    return results_df

In [ ]:
all_model_results = evaluate_all_models_ranking(
    train_data,
    test_data,
    user_similarity_matrix=user_similarity_matrix,
    item_similarity_matrix=item_similarity_matrix,
    mf_model=mf_model,
    bpr_model=bpr_model,
    k=10,
)

# Display the final comparison table
print("\n--- Final Model Comparison (Ranking Function) ---")
display(all_model_results)

# Task 3) Implement baselines for both rating prediction and ranking tasks, and perform experiments with those baselines

# Task 4) Analysis of recommendation models. Analyzing the coefficients of hybrid model and the success of recommendation models for different users' groups.

## 4.1 Coefficients Analysis

In [ ]:
hybrid_rating_coeff, hybrid_ranking_coeff = coefficients_scaled, hybrid_ranking_weights_df

### 4.1.1 Rating predictions

> We noticed that if we add BPR to the hybrid model, its coefficient is very small (close to zero). This is confirms the fact that BPR is optimized for ranking, not rating prediction, so its predicted scores do not align well with actual ratings. Therefore, the regression model assigns it a low weight when predicting ratings. Also the coefficient values do mirror the RMSE values:
- MF (Matrix Factorization): Best RMSE $\approx 0.96$
- UserKNN: Best RMSE $\approx 1.01$
- ItemKNN: Best RMSE $\approx 1.14$
- CB (Content-Based): Best RMSE $\approx 1.17$

In [ ]:
hybrid_rating_coeff

In [ ]:
plt.figure(figsize=(7,4), dpi=200)
sns.barplot(data=hybrid_rating_coeff, x='Component', y='Alpha (Weight)', palette='coolwarm')
plt.title('Hybrid Model Component Importance for Rating Prediction Task')
plt.ylabel('Weight (α)')
plt.show()

**Analysis**

- MF (0.36) dominates, as expected — it’s excellent at modeling user–item rating patterns.
- UserKNN (0.31) still provides complementary information about local neighborhood similarities.
- ItemKNN contributes almost nothing — likely redundant with MF and user-based CF.
- Content-based is negligible — explicit ratings are driven more by collaborative patterns than by item content.

**Conclusion**

For rating prediction, the hybrid model relies mainly on MF and UserKNN, with MF being slightly more influential.

### 4.1.2 Ranking predictions

In [ ]:
hybrid_ranking_coeff

In [ ]:
plt.figure(figsize=(7,4), dpi=200)
sns.barplot(data=hybrid_ranking_coeff, x='Component', y='Weight (α)', palette='coolwarm')
plt.title('Hybrid Model Component Importance for Ranking Task')
plt.ylabel('Weight (α)')
plt.show()

**Analysis**

- BPR (0.46) has the highest coefficient — consistent with its design as a ranking-optimized model (pairwise loss).
- MF (0.26) and UserKNN (0.29) provide secondary signals; both capture collaborative structure.
- Content-based (0.11) and ItemKNN (0.09) offer additional diversity (especially helpful for cold-start and diversity).

**Conclusion**

- For ranking tasks, the hybrid model relies primarily on BPR, followed by UserKNN and MF.
- Content-based and ItemKNN models make smaller but complementary contributions.

## 4.2 User Group Analysis

Group users by **activity level** (user behaviour length)

In [ ]:
def evaluate_by_user_group_ranking(
    train_data,
    test_data,
    user_similarity_matrix,
    item_similarity_matrix,
    mf_model,
    bpr_model,
    device="cpu",
    k_recall=10,
    user_bins=[0, 10, 50, 200, np.inf],
    k_user=70,
    k_item=20,
):
    """
    Evaluate ranking metrics (Precision@K, Recall@K, NDCG@K, MAP@K)
    for CB, UserKNN, ItemKNN, MF, and BPR models by user group.
    """

    results = []

    # 1 Split users by activity
    user_counts = train_data.groupby("user_id").size()
    user_groups = pd.cut(
        user_counts,
        bins=user_bins,
        labels=["cold", "light", "medium", "heavy"]
    )

    global_mean_rating = train_data["rating"].mean()
    all_items = test_data["item_id"].unique()

    # 2 Evaluate per group
    for group_name, group_users in user_groups.groupby(user_groups):
        group_user_ids = group_users.index.tolist()
        subset = test_data[test_data["user_id"].isin(group_user_ids)]
        if subset.empty:
            continue

        group_result = {"group": group_name}

        # 3 Evaluate each model
        for model_name in ["cb", "userknn", "itemknn", "mf", "bpr"]:
            precisions, recalls, ndcgs, maps = [], [], [], []

            for user_id in tqdm(group_user_ids, desc=f"Group {group_name} - {model_name}", leave=False):
                true_items = test_data[test_data["user_id"] == user_id]["item_id"].tolist()
                if len(true_items) == 0:
                    continue

                # Candidate items (exclude those seen in train)
                user_train_items = train_data[train_data["user_id"] == user_id]["item_id"].tolist()
                candidates = [i for i in all_items if i not in user_train_items]

                scores = []
                for item_id in candidates:
                    try:
                        if model_name == "cb":
                            score = get_user_item_prediction_content_based(
                                train_data, user_id, item_id, "full", "weighted_avg"
                            )
                        elif model_name == "userknn":
                            score = predict_rating_user_based(
                                train_data, user_similarity_matrix,
                                user_id, item_id, global_mean_rating, k=k_user
                            )
                        elif model_name == "itemknn":
                            score = predict_rating_item_based(
                                train_data, item_similarity_matrix,
                                user_id, item_id, k=k_item
                            )
                        elif model_name == "mf":
                            score = mf_model.predict(user_id, item_id)
                        elif model_name == "bpr":
                            with torch.no_grad():
                                score = bpr_model.model.predict(
                                    torch.LongTensor([user_id - 1]).to(device),
                                    torch.LongTensor([item_id - 1]).to(device)
                                ).item()
                        else:
                            continue
                        scores.append((item_id, score))
                    except Exception:
                        continue

                if len(scores) == 0:
                    continue

                ranked_items = [i for i, _ in sorted(scores, key=lambda x: x[1], reverse=True)]

                precisions.append(precision_at_k(true_items, ranked_items, k=k_recall))
                recalls.append(recall_at_k(true_items, ranked_items, k=k_recall))
                ndcgs.append(ndcg_at_k(true_items, ranked_items, k=k_recall))
                maps.append(average_precision_at_k(true_items, ranked_items, k=k_recall))

            # Aggregate metrics
            group_result[f"{model_name}_Precision@{k_recall}"] = np.nanmean(precisions)
            group_result[f"{model_name}_Recall@{k_recall}"] = np.nanmean(recalls)
            group_result[f"{model_name}_NDCG@{k_recall}"] = np.nanmean(ndcgs)
            group_result[f"{model_name}_MAP@{k_recall}"] = np.nanmean(maps)

        results.append(group_result)

    results_df = pd.DataFrame(results)
    print("Multi-metric ranking evaluation completed by user group.")
    return results_df

In [ ]:
"""====== MORE THAN 1 HOUR IS NEEDED TO RUN THE CODE BELOW ======"""
# results_multi_metric = evaluate_by_user_group_ranking(
#     train_data=train_data,
#     test_data=test_data,
#     user_similarity_matrix=user_similarity_matrix,
#     item_similarity_matrix=item_similarity_matrix,
#     mf_model=mf_model,
#     bpr_model=bpr_model,
#     device=device,
#     k_recall=10
# )

# results_multi_metric

In [ ]:
# results_multi_metric.to_csv(os.path.join(OUTPUTS_PATH, 'user_group_analysis.csv'))

In [ ]:
"""====== LOAD THE RESULTS INSTEAD ======"""
results_multi_metric = pd.read_csv(os.path.join(OUTPUTS_PATH, 'user_group_analysis.csv'))
results_multi_metric

In [ ]:
def plot_ranking_metrics_by_group(results_df, k=10, save_path=None):
    """
    Plot grouped bar charts for Precision@K, Recall@K, NDCG@K, and MAP@K.
    """
    metrics = [f"Precision@{k}", f"Recall@{k}", f"NDCG@{k}", f"MAP@{k}"]
    models = ["cb", "userknn", "itemknn", "mf", "bpr"]

    # Melt dataframe for plotting
    plot_data = []
    for _, row in results_df.iterrows():
        group = row["group"]
        for model in models:
            for metric in metrics:
                col = f"{model}_{metric}"
                if col in row:
                    plot_data.append({
                        "Group": group,
                        "Model": model.upper(),
                        "Metric": metric.replace(f"@{k}", ""),
                        "Value": row[col]
                    })

    df_plot = pd.DataFrame(plot_data)

    # Seaborn style setup
    sns.set(style="whitegrid", font_scale=1.1)
    plt.figure(figsize=(16, 10), dpi=400)

    # Plot 4 metrics separately
    for i, metric in enumerate(metrics, 1):
        plt.subplot(2, 2, i)
        subset = df_plot[df_plot["Metric"] == metric.replace(f"@{k}", "")]
        sns.barplot(data=subset, x="Group", y="Value", hue="Model")
        plt.title(metric)
        plt.ylabel("Score")
        plt.xlabel("User Group")
        plt.legend(title="Model", loc="upper right")

    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        print(f"✅ Plot saved to {save_path}")
    else:
        plt.show()

In [ ]:
plot_ranking_metrics_by_group(results_multi_metric, k=10)

**Conclusion**

In this case, **most metrics (Precision, NDCG, MAP)** show that recommendation accuracy increases with user activity —
**heavy users > medium > light > cold**,
meaning models perform best for users with more interactions, since their preferences are easier to learn.

However, **Recall** shows the opposite trend (**cold > light > medium > heavy**), indicating that although models make less precise predictions for cold users, they still retrieve a larger portion of relevant items — <u>possibly because cold users have fewer total relevant items, making it easier to achieve higher recall</u>.

👉 **Summary:**

* **Best performance (overall accuracy):** Heavy users
* **Highest recall:** Cold users

# Task 5) Evaluation of beyond accuracy

## Diversity (intra-list diversity)

**Intra-List Diversity (ILD)** quantifies how *different* the items within a single user’s recommendation list are from each other.

Formally:
[
ILD(u) = \frac{1}{|L_u|(|L_u|-1)} \sum_{i \ne j} (1 - \text{sim}(i,j))
]
where `sim(i,j)` is cosine similarity between item embeddings (here, BERT item representations).
Then we average across users to get the final ILD score.

* **High ILD (close to 1)** → items are *very dissimilar* → *diverse recommendations* (e.g., a mix of action, romance, and comedy movies).
* **Low ILD (close to 0)** → items are *very similar* → *homogeneous recommendations* (e.g., 10 action movies with similar plots).



### Define distance function

> We'll use cosine distance based on item content embeddings (BERT or title+genre).

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cosine_sim_matrix
import numpy as np
import pandas as pd
from tqdm import tqdm

def compute_intra_list_diversity(rec_lists, item_embeddings):
    """
    Compute average ILD across all users.

    Args:
        rec_lists (dict): {user_id: [item_ids]} – recommended items per user
        item_embeddings (np.ndarray): matrix of item embeddings (indexed by item_id - 1)
    Returns:
        float: average ILD score
    """
    ild_scores = []
    for user, rec_items in rec_lists.items():
        if len(rec_items) < 2:
            continue  # skip too-short lists
        embs = item_embeddings[[i - 1 for i in rec_items]]
        sim_matrix = cosine_sim_matrix(embs)
        # convert to distance
        dist_matrix = 1 - sim_matrix
        n = len(rec_items)
        ild = dist_matrix.sum() / (n * (n - 1))
        ild_scores.append(ild)
    return np.mean(ild_scores)


### Generate recommendation lists

In [ ]:
# Generate Top-10 lists for a random sample of users
sample_users = np.random.choice(train_data['user_id'].unique(), size=10, replace=False)

# topk_cb, topk_userknn, topk_itemknn, topk_mf, topk_bpr, topk_hybrid = {}, {}, {}, {}, {}, {}
topk_itemknn = {}
topk_hybrid = {}

for user in tqdm(sample_users):
    # topk_cb[user] = recommend_topk_item_based(train_data, item_similarity_matrix, user, k=10)
    # topk_userknn[user] = [i for i, _ in recommend_topk_user_based(train_data, user_similarity_matrix, user, k=10)]
    topk_itemknn[user] = [i for i, _ in recommend_topk_item_based(train_data, item_similarity_matrix, user, k_item=20, k=10)]
    # topk_mf[user] = [i for i, _ in recommend_topk_MF(mf_model, train_data, user, k=10)]
    # topk_bpr[user] = [i for i, _ in bpr_model.recommend(user, train_data=train_data, top_k=10)]
    topk_hybrid[user] = [
    i for i, _ in recommend_topk_hybrid_ranking(
        user,
        hybrid_reg_scaled,
        train_data,
        item_similarity_matrix,
        user_similarity_matrix,
        mf_model,
        bpr_model,
        k_item=20, 
        k_user=70, 
        top_k=10, 
        device=device
    )
]


### Compute ILD per model

In [ ]:
emb_to_use = item_emb_full  # or item_emb_titlegenres

# ild_cb = compute_intra_list_diversity(topk_cb, emb_to_use)
# ild_userknn = compute_intra_list_diversity(topk_userknn, emb_to_use)
ild_itemknn = compute_intra_list_diversity(topk_itemknn, emb_to_use)
# ild_mf = compute_intra_list_diversity(topk_mf, emb_to_use)
# ild_bpr = compute_intra_list_diversity(topk_bpr, emb_to_use)
ild_hybrid = compute_intra_list_diversity(topk_hybrid, emb_to_use)

results_ild = pd.DataFrame({
    'Model': ['CB', 'UserKNN', 'ItemKNN', 'MF', 'BPR', 'Hybrid'],
    'Intra-List Diversity': [None, None, ild_itemknn, None, None, ild_hybrid]
})
display(results_ild)

### Visualize diversity comparison

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.barplot(data=results_ild, x='Model', y='Intra-List Diversity', palette='viridis')
plt.title('Intra-List Diversity (ILD) Comparison Across Models')
plt.ylabel('Average ILD')
plt.xlabel('Recommendation Model')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.show()

## Novelty (surprisal)

Measure each item’s novelty as:

[
\text{novelty}(i)=-\log_2 p(i)
\quad\text{with}\quad
p(i)=\frac{#\text{interactions on }i}{\text{total interactions}}
]

A user’s novelty is the **mean** surprisal of the items in their recommended list, and finally average across users.

### Item popularity

In [ ]:
def build_item_popularity(train_df):
    """
    Returns:
      pop: pd.Series indexed by item_id with p(i) in [0,1]
    """
    item_counts = train_df['item_id'].value_counts().sort_index()
    total = item_counts.sum()
    pop = item_counts / total
    # also ensure every known item appears (even if 0 in train subset)
    all_items = train_df['item_id'].unique()
    pop = pop.reindex(sorted(all_items), fill_value=0.0)
    return pop

### Item surprisal

In [ ]:
def build_item_surprisal(pop, eps=1e-12):
    """
    novelty(i) = -log2(pop(i) + eps)  (in bits)
    """
    return -np.log2(pop + eps)

### Example build from training split

In [ ]:
item_pop = build_item_popularity(train_data)
item_surprisal = build_item_surprisal(item_pop)

print("Example surprisal (higher = more novel):")
display(item_surprisal.sample(5).to_frame("novelty_bits"))

## Calibration

## Fairness metrics

### User Side Fairness

#### Group Recommendation Unfairness (GRU)

### Item Side Fairness

#### Demographic Parity

#### Disparate treatment

#### Disparate Impact

# Bibliography